# Chapter 8: Advanced Data Types

SystemVerilog provides powerful data types that go far beyond the basic wire and reg types found in traditional Verilog. These advanced data types enable more efficient modeling, better code organization, and improved verification capabilities.

## Dynamic Arrays

Dynamic arrays allow you to create arrays whose size can be determined and modified at runtime, providing flexibility for handling variable amounts of data.

### Declaration and Basic Usage

```systemverilog
// Dynamic array declaration
int dynamic_array[];

// Allocating memory
initial begin
    dynamic_array = new[10]; // Create array with 10 elements
    
    // Initialize values
    foreach(dynamic_array[i]) begin
        dynamic_array[i] = i * 2;
    end
    
    // Display values
    $display("Array size: %0d", dynamic_array.size());
    foreach(dynamic_array[i]) begin
        $display("dynamic_array[%0d] = %0d", i, dynamic_array[i]);
    end
end
```

### Dynamic Array Methods

```systemverilog
module dynamic_array_methods;
    int data[];
    
    initial begin
        // Allocate initial array
        data = new[5];
        
        // Initialize with values
        data = '{10, 20, 30, 40, 50};
        
        $display("Original array size: %0d", data.size());
        
        // Delete array (deallocate memory)
        data.delete();
        $display("After delete, size: %0d", data.size());
        
        // Reallocate with different size
        data = new[8];
        $display("New array size: %0d", data.size());
        
        // Copy from another array
        int source[] = '{1, 2, 3, 4};
        data = new[source.size()](source);
        
        $display("Copied array:");
        foreach(data[i]) $display("data[%0d] = %0d", i, data[i]);
    end
endmodule
```

### Multidimensional Dynamic Arrays

```systemverilog
// 2D dynamic array
int matrix[][];

initial begin
    // Allocate 3x4 matrix
    matrix = new[3];
    foreach(matrix[i]) begin
        matrix[i] = new[4];
    end
    
    // Initialize matrix
    foreach(matrix[i]) begin
        foreach(matrix[i][j]) begin
            matrix[i][j] = i * 4 + j;
        end
    end
    
    // Display matrix
    foreach(matrix[i]) begin
        foreach(matrix[i][j]) begin
            $write("%3d ", matrix[i][j]);
        end
        $display("");
    end
end
```

### **Example 1: Basic Dynamic Array Creation**
Simple example showing how to declare, allocate, and initialize a dynamic array with basic operations.

In [1]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_1__dynamic_array_basics/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// dynamic_array_processor.sv
module dynamic_array_processor;              // Design under test
  
  // Dynamic array declarations
  int dynamic_numbers[];                      // Integer dynamic array
  string dynamic_words[];                     // String dynamic array
  
  initial begin
    $display();                               // Display empty line
    $display("=== Dynamic Array Processor ===");
    
    // Allocate and initialize integer array
    dynamic_numbers = new[5];                 // Allocate 5 elements
    dynamic_numbers[0] = 10;
    dynamic_numbers[1] = 20;
    dynamic_numbers[2] = 30;
    dynamic_numbers[3] = 40;
    dynamic_numbers[4] = 50;
    
    $display("Integer array size: %0d", dynamic_numbers.size());
    $display("Integer array contents: %p", dynamic_numbers);
    
    // Allocate and initialize string array
    dynamic_words = new[3];                   // Allocate 3 elements
    dynamic_words[0] = "Hello";
    dynamic_words[1] = "Dynamic";
    dynamic_words[2] = "Arrays";
    
    $display("String array size: %0d", dynamic_words.size());
    $display("String array contents: %p", dynamic_words);
    
    // Resize operations
    dynamic_numbers = new[8](dynamic_numbers); // Resize to 8, keep data
    dynamic_numbers[5] = 60;
    dynamic_numbers[6] = 70;
    dynamic_numbers[7] = 80;
    
    $display("After resize - Integer array size: %0d", 
             dynamic_numbers.size());
    $display("After resize - Integer array: %p", dynamic_numbers);
    
    $display("=== Design Processing Complete ===");
  end
  
endmodule
```

```systemverilog
// dynamic_array_processor_testbench.sv
module dynamic_array_testbench;              // Testbench module
  
  // Instantiate design under test
  dynamic_array_processor PROCESSOR_INSTANCE();
  
  // Testbench dynamic arrays for verification
  int verification_array[];
  bit test_results[];
  
  initial begin
    // Dump waves
    $dumpfile("dynamic_array_testbench.vcd"); // Specify the VCD file
    $dumpvars(0, dynamic_array_testbench);    // Dump all variables
    
    #1;                                       // Wait for design to complete
    
    $display();                               // Display empty line
    $display("=== Testbench Verification ===");
    
    // Create verification array with different operations
    verification_array = new[4];             // Allocate 4 elements
    verification_array = '{100, 200, 300, 400}; // Initialize with literal
    
    // Display verification array size and contents
    $display("Verification array: %p", verification_array);
    
    // Test dynamic allocation and deallocation
    test_results = new[3];                    // Allocate test results
    test_results[0] = (verification_array.size() == 4);
    test_results[1] = (verification_array[0] == 100);
    test_results[2] = (verification_array[3] == 400);
    
    $display("Test results: %p", test_results);
    
    // Demonstrate delete operation
    verification_array.delete();             // Delete all elements
    $display("After delete - size: %0d", verification_array.size());
    
    $display("=== Testbench Verification Complete ===");
    $display();                               // Display empty line
    
    #10;                                      // Final delay
    $finish;                                  // End simulation
  end
  
endmodule
```

Verilator Simulation Output:
=== Design Processing Complete ===

=== Testbench Verification ===
Verification array: '{'h64, 'hc8, 'h12c, 'h190}
Test results: '{'h1, 'h1, 'h1}
After delete - size: 0
=== Testbench Verification Complete ===

Process finished with return code: 0
Removing Chapter_8_examples/example_1__dynamic_array_basics/obj_dir directory...
Chapter_8_examples/example_1__dynamic_array_basics/obj_dir removed successfully.


0

### **Example 2: Growing and Shrinking Arrays**
Demonstrates resizing dynamic arrays at runtime - making them bigger or smaller based on needs.

In [2]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_2__growing_shrinking_arrays/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// array_manager.sv
module array_manager();               // Dynamic array manager module
  
  // Dynamic arrays for demonstration
  int data_storage[];                 // Main data storage array
  string name_list[];                 // Array of names
  
  // Task to grow array and add new elements
  task automatic grow_data_array(int new_size, int fill_value);
    int old_size = data_storage.size();
    data_storage = new[new_size](data_storage);  // Grow array
    
    // Fill new elements with fill_value
    for (int i = old_size; i < new_size; i++) begin
      data_storage[i] = fill_value;
    end
    
    $display("Array grown from %0d to %0d elements", old_size, new_size);
  endtask
  
  // Task to shrink array by removing elements from the end
  task automatic shrink_data_array(int new_size);
    int old_size = data_storage.size();
    if (new_size < old_size) begin
      data_storage = new[new_size](data_storage);  // Shrink array
      $display("Array shrunk from %0d to %0d elements", old_size, new_size);
    end else begin
      $display("Cannot shrink: new size %0d >= current size %0d", 
               new_size, old_size);
    end
  endtask
  
  // Task to display current array contents
  task automatic display_array_contents();
    $display("Current array size: %0d", data_storage.size());
    if (data_storage.size() > 0) begin
      $write("Contents: ");
      foreach (data_storage[i]) begin
        $write("%0d ", data_storage[i]);
      end
      $display();
    end else begin
      $display("Array is empty");
    end
  endtask
  
  // Task to manage name list array
  task automatic manage_name_list();
    // Start with small array
    name_list = new[2];
    name_list[0] = "Alice";
    name_list[1] = "Bob";
    $display("Initial name list: %p", name_list);
    
    // Grow to add more names
    name_list = new[4](name_list);
    name_list[2] = "Charlie";
    name_list[3] = "Diana";
    $display("Expanded name list: %p", name_list);
    
    // Shrink back to 3 names
    name_list = new[3](name_list);
    $display("Trimmed name list: %p", name_list);
  endtask

endmodule
```

```systemverilog
// array_manager_testbench.sv
module array_manager_testbench;      // Testbench for dynamic array operations
  
  // Instantiate the array manager
  array_manager ARRAY_MANAGER_INSTANCE();
  
  initial begin
    // Dump waves
    $dumpfile("array_manager_testbench.vcd");
    $dumpvars(0, array_manager_testbench);
    
    $display("=== Dynamic Array Growing and Shrinking Demo ===");
    $display();
    
    // Test 1: Start with empty array
    $display("Test 1: Starting with empty array");
    ARRAY_MANAGER_INSTANCE.display_array_contents();
    $display();
    
    // Test 2: Grow array to size 5 and fill with value 10
    $display("Test 2: Growing array to size 5");
    ARRAY_MANAGER_INSTANCE.grow_data_array(5, 10);
    ARRAY_MANAGER_INSTANCE.display_array_contents();
    $display();
    
    // Test 3: Grow array further to size 8 and fill with value 20
    $display("Test 3: Growing array to size 8");
    ARRAY_MANAGER_INSTANCE.grow_data_array(8, 20);
    ARRAY_MANAGER_INSTANCE.display_array_contents();
    $display();
    
    // Test 4: Shrink array to size 6
    $display("Test 4: Shrinking array to size 6");
    ARRAY_MANAGER_INSTANCE.shrink_data_array(6);
    ARRAY_MANAGER_INSTANCE.display_array_contents();
    $display();
    
    // Test 5: Try to shrink to larger size (should fail)
    $display("Test 5: Attempting to 'shrink' to size 10");
    ARRAY_MANAGER_INSTANCE.shrink_data_array(10);
    ARRAY_MANAGER_INSTANCE.display_array_contents();
    $display();
    
    // Test 6: Shrink to size 2
    $display("Test 6: Shrinking array to size 2");
    ARRAY_MANAGER_INSTANCE.shrink_data_array(2);
    ARRAY_MANAGER_INSTANCE.display_array_contents();
    $display();
    
    // Test 7: Demonstrate string array management
    $display("Test 7: String array growing and shrinking");
    ARRAY_MANAGER_INSTANCE.manage_name_list();
    $display();
    
    #1;  // Wait for a time unit
    $display("=== Array Management Demo Complete ===");
    
  end

endmodule
```

Verilator Simulation Output:
=== Dynamic Array Growing and Shrinking Demo ===

Test 1: Starting with empty array
Current array size: 0
Array is empty

Test 2: Growing array to size 5
Array grown from 0 to 5 elements
Current array size: 5
Contents: 10 10 10 10 10

Test 3: Growing array to size 8
Array grown from 5 to 8 elements
Current array size: 8
Contents: 10 10 10 10 10 20 20 20

Test 4: Shrinking array to size 6
Array shrunk from 8 to 6 elements
Current array size: 6
Contents: 10 10 10 10 10 20

Test 5: Attempting to 'shrink' to size 10
Cannot shrink: new size 10 >= current size 6
Current array size: 6
Contents: 10 10 10 10 10 20

Test 6: Shrinking array to size 2
Array shrunk from 6 to 2 elements
Current array size: 2
Contents: 10 10

Test 7: String array growing and shrinking
Initial name list: '{"Alice", "Bob"}
Expanded name list: '{"Alice", "Bob", "Charlie", "Diana"}
Trimmed name list: '{"Alice", "Bob", "Charlie"}

Process finished with return code: 0
Removing Chapter_8_example

0

### **Example 3: 2D Dynamic Grid**
Creates a simple dynamic 2D array (like a game board) that can change size during simulation.

In [3]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_3__dynamic_grid/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// game_board_grid.sv
module game_board_controller ();
  
  // Dynamic 2D array representing a game board
  int game_grid[][];
  int board_width, board_height;
  
  // Initialize the game board with given dimensions
  function automatic void initialize_board(int width, int height);
    board_width = width;
    board_height = height;
    
    // Allocate the 2D dynamic array
    game_grid = new[height];
    for (int row = 0; row < height; row++) begin
      game_grid[row] = new[width];
      // Initialize all cells to 0 (empty)
      for (int col = 0; col < width; col++) begin
        game_grid[row][col] = 0;
      end
    end
    
    $display("Board initialized: %0dx%0d grid", width, height);
  endfunction
  
  // Resize the board while preserving existing data
  function automatic void resize_board(int new_width, int new_height);
    int old_grid[][];
    int old_width = board_width;
    int old_height = board_height;
    
    // Save current grid
    old_grid = new[old_height];
    for (int row = 0; row < old_height; row++) begin
      old_grid[row] = new[old_width];
      for (int col = 0; col < old_width; col++) begin
        old_grid[row][col] = game_grid[row][col];
      end
    end
    
    // Create new grid
    board_width = new_width;
    board_height = new_height;
    game_grid = new[new_height];
    
    for (int row = 0; row < new_height; row++) begin
      game_grid[row] = new[new_width];
      for (int col = 0; col < new_width; col++) begin
        // Copy old data if within bounds, otherwise initialize to 0
        if (row < old_height && col < old_width) begin
          game_grid[row][col] = old_grid[row][col];
        end else begin
          game_grid[row][col] = 0;
        end
      end
    end
    
    $display("Board resized: %0dx%0d -> %0dx%0d", 
             old_width, old_height, new_width, new_height);
  endfunction
  
  // Place a piece on the board (1 = player piece)
  function automatic void place_piece(int row, int col, int piece_value);
    if (row >= 0 && row < board_height && col >= 0 && col < board_width) begin
      game_grid[row][col] = piece_value;
      $display("Placed piece %0d at position [%0d][%0d]", 
               piece_value, row, col);
    end else begin
      $display("Invalid position [%0d][%0d] for %0dx%0d board", 
               row, col, board_width, board_height);
    end
  endfunction
  
  // Display the current game board
  function automatic void display_board();
    $display("Current game board (%0dx%0d):", board_width, board_height);
    for (int row = 0; row < board_height; row++) begin
      string row_str = "";
      for (int col = 0; col < board_width; col++) begin
        row_str = {row_str, $sformatf("%2d ", game_grid[row][col])};
      end
      $display("  %s", row_str);
    end
    $display();
  endfunction
  
  initial begin
    $display("2D Dynamic Grid - Game Board Controller");
    $display("=====================================");
  end

endmodule
```

```systemverilog
// game_board_grid_testbench.sv
module dynamic_grid_testbench;
  
  // Instantiate the game board controller
  game_board_controller BOARD_CONTROLLER();
  
  initial begin
    // Dump waves for analysis
    $dumpfile("dynamic_grid_testbench.vcd");
    $dumpvars(0, dynamic_grid_testbench);
    
    $display("Testing 2D Dynamic Grid Operations");
    $display("==================================");
    $display();
    
    // Test 1: Initialize a small 3x3 game board
    $display("Test 1: Creating initial 3x3 board");
    BOARD_CONTROLLER.initialize_board(3, 3);
    BOARD_CONTROLLER.display_board();
    
    // Test 2: Place some game pieces
    $display("Test 2: Placing game pieces");
    BOARD_CONTROLLER.place_piece(0, 0, 1);  // Player 1 piece
    BOARD_CONTROLLER.place_piece(1, 1, 2);  // Player 2 piece
    BOARD_CONTROLLER.place_piece(2, 2, 1);  // Player 1 piece
    BOARD_CONTROLLER.display_board();
    
    // Test 3: Resize to larger board (5x4)
    $display("Test 3: Expanding board to 5x4");
    BOARD_CONTROLLER.resize_board(5, 4);
    BOARD_CONTROLLER.display_board();
    
    // Test 4: Add more pieces to expanded area
    $display("Test 4: Adding pieces to new areas");
    BOARD_CONTROLLER.place_piece(3, 4, 2);  // New area
    BOARD_CONTROLLER.place_piece(0, 3, 1);  // New column
    BOARD_CONTROLLER.place_piece(3, 0, 2);  // New row
    BOARD_CONTROLLER.display_board();
    
    // Test 5: Resize to smaller board (2x2) - some data will be lost
    $display("Test 5: Shrinking board to 2x2");
    BOARD_CONTROLLER.resize_board(2, 2);
    BOARD_CONTROLLER.display_board();
    
    // Test 6: Try invalid placement
    $display("Test 6: Testing boundary checking");
    BOARD_CONTROLLER.place_piece(2, 2, 3);  // Should fail (out of bounds)
    BOARD_CONTROLLER.place_piece(1, 0, 3);  // Should succeed
    BOARD_CONTROLLER.display_board();
    
    // Test 7: Resize to different aspect ratio (6x2)
    $display("Test 7: Creating wide board (6x2)");
    BOARD_CONTROLLER.resize_board(6, 2);
    BOARD_CONTROLLER.place_piece(0, 5, 4);  // Far right
    BOARD_CONTROLLER.place_piece(1, 2, 4);  // Middle
    BOARD_CONTROLLER.display_board();
    
    $display("Dynamic grid testing completed!");
    $display();
    
    #1;  // Small delay for simulation
    $finish;
  end

endmodule
```

Verilator Simulation Output:
2D Dynamic Grid - Game Board Controller
Testing 2D Dynamic Grid Operations

Test 1: Creating initial 3x3 board
Board initialized: 3x3 grid
Current game board (3x3):
   0  0  0
   0  0  0
   0  0  0

Test 2: Placing game pieces
Placed piece 1 at position [0][0]
Placed piece 2 at position [1][1]
Placed piece 1 at position [2][2]
Current game board (3x3):
   1  0  0
   0  2  0
   0  0  1

Test 3: Expanding board to 5x4
Board resized: 3x3 -> 5x4
Current game board (5x4):
   1  0  0  0  0
   0  2  0  0  0
   0  0  1  0  0
   0  0  0  0  0

Test 4: Adding pieces to new areas
Placed piece 2 at position [3][4]
Placed piece 1 at position [0][3]
Placed piece 2 at position [3][0]
Current game board (5x4):
   1  0  0  1  0
   0  2  0  0  0
   0  0  1  0  0
   2  0  0  0  2

Test 5: Shrinking board to 2x2
Board resized: 5x4 -> 2x2
Current game board (2x2):
   1  0
   0  2

Test 6: Testing boundary checking
Invalid position [2][2] for 2x2 board
Placed piece 3 at position

0

## Associative Arrays

Associative arrays store elements in a sparse manner, indexed by any data type rather than consecutive integers. They're ideal for lookup tables and sparse data structures.

### Basic Associative Arrays

```systemverilog
module associative_arrays;
    // String-indexed associative array
    int lookup_table[string];
    
    // Bit-indexed associative array
    string name_table[bit[31:0]];
    
    initial begin
        // Populate string-indexed array
        lookup_table["apple"] = 100;
        lookup_table["banana"] = 200;
        lookup_table["cherry"] = 300;
        
        // Access elements
        $display("apple value: %0d", lookup_table["apple"]);
        
        // Check if key exists
        if (lookup_table.exists("grape")) begin
            $display("Grape found: %0d", lookup_table["grape"]);
        end else begin
            $display("Grape not found");
        end
        
        // Populate bit-indexed array
        name_table[32'h1001] = "Alice";
        name_table[32'h2002] = "Bob";
        name_table[32'h3003] = "Charlie";
        
        $display("ID 0x1001: %s", name_table[32'h1001]);
    end
endmodule
```

### Associative Array Methods

```systemverilog
module assoc_array_methods;
    int grades[string];
    string student_names[$];
    int grade_values[$];
    
    initial begin
        // Populate array
        grades["Alice"] = 85;
        grades["Bob"] = 92;
        grades["Charlie"] = 78;
        grades["Diana"] = 96;
        
        $display("Number of students: %0d", grades.num());
        
        // Get all keys and values
        grades.first(student_names);
        grades.second(grade_values);
        
        $display("\nAll students and grades:");
        for (int i = 0; i < student_names.size(); i++) begin
            $display("%s: %0d", student_names[i], grade_values[i]);
        end
        
        // Iterate through array
        string name;
        $display("\nUsing next() method:");
        if (grades.first(name)) begin
            do begin
                $display("%s: %0d", name, grades[name]);
            end while (grades.next(name));
        end
        
        // Delete specific entry
        grades.delete("Bob");
        $display("\nAfter deleting Bob, size: %0d", grades.num());
    end
endmodule
```

### **Example 4: Student Grade Lookup**
Basic string-indexed associative array storing student names as keys and grades as values.

In [4]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_4__student_grade_lookup/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// student_grade_storage.sv
module student_grade_manager ();
  // String-indexed associative array for student grades
  int student_grades[string];
  
  // Task to add or update student grade
  task add_student_grade(input string student_name, input int grade);
    student_grades[student_name] = grade;
    $display("Added/Updated: %s = %0d", student_name, grade);
  endtask
  
  // Function to lookup student grade
  function int lookup_student_grade(input string student_name);
    if (student_grades.exists(student_name) == 1) begin
      return student_grades[student_name];
    end else begin
      $display("Student '%s' not found in database", student_name);
      return -1;  // Return -1 for not found
    end
  endfunction
  
  // Task to display all students and grades
  task display_all_grades();
    string student_name;
    $display("\n=== All Student Grades ===");
    if (student_grades.first(student_name) == 1) begin
      do begin
        $display("%-15s: %0d", student_name, student_grades[student_name]);
      end while (student_grades.next(student_name) == 1);
    end else begin
      $display("No students in database");
    end
    $display("========================\n");
  endtask
  
  // Task to remove student from database
  task remove_student(input string student_name);
    if (student_grades.exists(student_name) == 1) begin
      student_grades.delete(student_name);
      $display("Removed student: %s", student_name);
    end else begin
      $display("Cannot remove - Student '%s' not found", student_name);
    end
  endtask
  
  // Function to get total number of students
  function int get_student_count();
    return student_grades.size();
  endfunction

endmodule
```

```systemverilog
// student_grade_storage_testbench.sv
module grade_lookup_testbench;
  // Instantiate the student grade manager
  student_grade_manager GRADE_MANAGER_INSTANCE();
  
  int found_grade;
  
  initial begin
    // Dump waves
    $dumpfile("grade_lookup_testbench.vcd");
    $dumpvars(0, grade_lookup_testbench);
    
    $display("=== Student Grade Lookup System Test ===\n");
    
    // Test 1: Add students and grades
    $display("Test 1: Adding student grades");
    GRADE_MANAGER_INSTANCE.add_student_grade("Alice Johnson", 95);
    GRADE_MANAGER_INSTANCE.add_student_grade("Bob Smith", 87);
    GRADE_MANAGER_INSTANCE.add_student_grade("Carol Williams", 92);
    GRADE_MANAGER_INSTANCE.add_student_grade("David Brown", 78);
    GRADE_MANAGER_INSTANCE.add_student_grade("Emma Davis", 89);
    $display();
    
    // Test 2: Display all grades
    $display("Test 2: Display all student grades");
    GRADE_MANAGER_INSTANCE.display_all_grades();
    
    // Test 3: Lookup existing students
    $display("Test 3: Looking up existing students");
    found_grade = GRADE_MANAGER_INSTANCE.lookup_student_grade("Alice Johnson");
    $display("Alice Johnson's grade: %0d", found_grade);
    
    found_grade = GRADE_MANAGER_INSTANCE.lookup_student_grade("David Brown");
    $display("David Brown's grade: %0d", found_grade);
    $display();
    
    // Test 4: Lookup non-existing student
    $display("Test 4: Looking up non-existing student");
    found_grade = GRADE_MANAGER_INSTANCE.lookup_student_grade("John Doe");
    $display();
    
    // Test 5: Update existing student grade
    $display("Test 5: Updating existing student grade");
    GRADE_MANAGER_INSTANCE.add_student_grade("Bob Smith", 91);
    found_grade = GRADE_MANAGER_INSTANCE.lookup_student_grade("Bob Smith");
    $display("Bob Smith's updated grade: %0d", found_grade);
    $display();
    
    // Test 6: Check student count
    $display("Test 6: Total student count");
    $display("Number of students: %0d", 
             GRADE_MANAGER_INSTANCE.get_student_count());
    $display();
    
    // Test 7: Remove a student
    $display("Test 7: Removing a student");
    GRADE_MANAGER_INSTANCE.remove_student("Carol Williams");
    $display("After removal - Number of students: %0d",
             GRADE_MANAGER_INSTANCE.get_student_count());
    $display();
    
    // Test 8: Try to remove non-existing student
    $display("Test 8: Trying to remove non-existing student");
    GRADE_MANAGER_INSTANCE.remove_student("Jane Doe");
    $display();
    
    // Test 9: Final display of all grades
    $display("Test 9: Final state of student database");
    GRADE_MANAGER_INSTANCE.display_all_grades();
    
    $display("=== Test Complete ===");
    #1;
  end

endmodule
```

Verilator Simulation Output:
=== Student Grade Lookup System Test ===

Test 1: Adding student grades
Added/Updated: Alice Johnson = 95
Added/Updated: Bob Smith = 87
Added/Updated: Carol Williams = 92
Added/Updated: David Brown = 78
Added/Updated: Emma Davis = 89

Test 2: Display all student grades

=== All Student Grades ===
Alice Johnson  : 95
Bob Smith      : 87
Carol Williams : 92
David Brown    : 78
Emma Davis     : 89

Test 3: Looking up existing students
Alice Johnson's grade: 95
David Brown's grade: 78

Test 4: Looking up non-existing student
Student 'John Doe' not found in database

Test 5: Updating existing student grade
Added/Updated: Bob Smith = 91
Bob Smith's updated grade: 91

Test 6: Total student count
Number of students: 5

Test 7: Removing a student
Removed student: Carol Williams
After removal - Number of students: 4

Test 8: Trying to remove non-existing student
Cannot remove - Student 'Jane Doe' not found

Test 9: Final state of student database

=== All Student Gra

0

### **Example 5: Color Code Dictionary**
Simple lookup table mapping color names to RGB hex values using associative arrays.

In [5]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_5__color_code_dictionary/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// color_lookup_dictionary.sv
module color_lookup_dictionary ();
  // RGB color code type definition (24-bit hex value)
  typedef logic [23:0] rgb_color_t;
  
  // Associative array: color name string -> RGB hex value
  rgb_color_t color_dictionary[string];
  
  initial begin
    // Populate the color dictionary with common colors
    color_dictionary["red"]     = 24'hFF0000;  // Red
    color_dictionary["green"]   = 24'h00FF00;  // Green  
    color_dictionary["blue"]    = 24'h0000FF;  // Blue
    color_dictionary["white"]   = 24'hFFFFFF;  // White
    color_dictionary["black"]   = 24'h000000;  // Black
    color_dictionary["yellow"]  = 24'hFFFF00;  // Yellow
    color_dictionary["cyan"]    = 24'h00FFFF;  // Cyan
    color_dictionary["magenta"] = 24'hFF00FF;  // Magenta
    color_dictionary["orange"]  = 24'hFFA500;  // Orange
    color_dictionary["purple"]  = 24'h800080;  // Purple
    
    $display();
    $display("Color Dictionary Initialized!");
    $display("Dictionary contains %0d colors", color_dictionary.size());
  end
  
  // Function to lookup color by name
  function rgb_color_t lookup_color_by_name(string color_name);
    if (|color_dictionary.exists(color_name)) begin
      return color_dictionary[color_name];
    end else begin
      $display("Warning: Color '%s' not found in dictionary!", color_name);
      return 24'h000000; // Return black as default
    end
  endfunction
  
  // Function to display all colors in dictionary
  function void display_all_colors();
    string color_name;
    rgb_color_t rgb_value;
    
    $display();
    $display("=== Color Dictionary Contents ===");
    if (|color_dictionary.first(color_name)) begin
      do begin
        rgb_value = color_dictionary[color_name];
        $display("%-8s -> #%06X", color_name, rgb_value);
      end while (|color_dictionary.next(color_name));
    end
    $display("=================================");
  endfunction

endmodule
```

```systemverilog
// color_lookup_dictionary_testbench.sv
module color_lookup_dictionary_testbench;
  // Instantiate the color dictionary design under test
  color_lookup_dictionary COLOR_DICT_INSTANCE();
  
  // Test variables
  logic [23:0] retrieved_color;
  string test_colors[] = {"red", "blue", "yellow", "pink", "green"};
  
  initial begin
    // Setup wave dump
    $dumpfile("color_lookup_dictionary_testbench.vcd");
    $dumpvars(0, color_lookup_dictionary_testbench);
    
    // Wait for dictionary initialization
    #2;
    
    $display();
    $display("Starting Color Dictionary Tests...");
    
    // Display all available colors
    COLOR_DICT_INSTANCE.display_all_colors();
    
    $display();
    $display("=== Color Lookup Tests ===");
    
    // Test color lookups
    foreach (test_colors[i]) begin
      retrieved_color = COLOR_DICT_INSTANCE.lookup_color_by_name(
                                                            test_colors[i]);
      if (|COLOR_DICT_INSTANCE.color_dictionary.exists(test_colors[i])) begin
        $display("Found '%s' -> #%06X", test_colors[i], retrieved_color);
      end else begin
        $display("'%s' not found, returned default: #%06X", 
                 test_colors[i], retrieved_color);
      end
      #1;
    end
    
    $display();
    $display("=== Adding New Colors ===");
    
    // Add new colors to dictionary during simulation
    COLOR_DICT_INSTANCE.color_dictionary["pink"] = 24'hFFC0CB;
    COLOR_DICT_INSTANCE.color_dictionary["lime"] = 24'h00FF00;
    
    $display("Added 'pink' and 'lime' to dictionary");
    $display("Dictionary now contains %0d colors", 
             COLOR_DICT_INSTANCE.color_dictionary.size());
    
    // Test newly added colors
    retrieved_color = COLOR_DICT_INSTANCE.lookup_color_by_name("pink");
    $display("Found 'pink' -> #%06X", retrieved_color);
    
    $display();
    $display("Hello from color dictionary testbench!");
    $display();
    
    #5;
    $finish;
  end

endmodule
```

Verilator Simulation Output:

Color Dictionary Initialized!
Dictionary contains 10 colors

Starting Color Dictionary Tests...

=== Color Dictionary Contents ===
black    -> #0
blue     -> #ff
cyan     -> #ffff
green    -> #ff00
magenta  -> #ff00ff
orange   -> #ffa500
purple   -> #800080
red      -> #ff0000
white    -> #ffffff
yellow   -> #ffff00

=== Color Lookup Tests ===
Found 'red' -> #ff0000
Found 'blue' -> #0000ff
Found 'yellow' -> #ffff00
'pink' not found, returned default: #000000
Found 'green' -> #00ff00

=== Adding New Colors ===
Added 'pink' and 'lime' to dictionary
Dictionary now contains 12 colors
Found 'pink' -> #ffc0cb

Hello from color dictionary testbench!

Process finished with return code: 0
Removing Chapter_8_examples/example_5__color_code_dictionary/obj_dir directory...
Chapter_8_examples/example_5__color_code_dictionary/obj_dir removed successfully.


0

### **Example 6: Sparse Memory Model**
Demonstrates using associative arrays to model memory where only some addresses contain data.

In [6]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_6__sparse_memory_model/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// sparse_memory_controller.sv
module sparse_memory_controller ();

  // Sparse memory using associative array - only stores written addresses
  logic [31:0] sparse_memory_data [logic [31:0]];
  
  // Memory interface signals
  logic [31:0] memory_address;
  logic [31:0] write_data;
  logic [31:0] read_data;
  logic        write_enable;
  logic        read_enable;
  logic        data_valid;

  // Keep track of written addresses using a separate array
  logic written_addresses [logic [31:0]];

  // Memory write operation - triggered by write_enable
  always @(posedge write_enable) begin
    if (write_enable) begin
      sparse_memory_data[memory_address] = write_data;
      written_addresses[memory_address] = 1'b1;
      $display("WRITE: Address 0x%08h = 0x%08h", 
               memory_address, write_data);
    end
  end

  // Memory read operation - combinational read
  always_comb begin
    read_data = 32'hzzzz_zzzz;
    data_valid = 1'b0;
    
    if (read_enable) begin
      if (written_addresses[memory_address] == 1'b1) begin
        read_data = sparse_memory_data[memory_address];
        data_valid = 1'b1;
      end else begin
        read_data = 32'h0000_0000;  // Default value for unwritten addresses
        data_valid = 1'b0;
      end
    end
  end

  // Display memory statistics
  task automatic display_memory_statistics();
    int used_addresses;
    
    used_addresses = 0;
    
    // Count used addresses
    foreach (written_addresses[addr]) begin
      if (written_addresses[addr] == 1'b1) begin
        used_addresses++;
      end
    end
    
    $display("Memory Statistics: %0d addresses in use", used_addresses);
    
    // Show all stored addresses
    if (used_addresses > 0) begin
      $display("Stored addresses:");
      foreach (written_addresses[addr]) begin
        if (written_addresses[addr] == 1'b1) begin
          $display("  0x%08h: 0x%08h", addr, sparse_memory_data[addr]);
        end
      end
    end
  endtask

endmodule
```

```systemverilog
// sparse_memory_controller_testbench.sv
module sparse_memory_testbench;

  // Instantiate the sparse memory controller
  sparse_memory_controller memory_controller_instance();

  // Test addresses - sparse pattern to demonstrate efficiency
  logic [31:0] test_addresses [5] = '{
    32'h0000_1000,  // Low address
    32'h8000_0000,  // High address  
    32'hFFFF_F000,  // Very high address
    32'h0000_2000,  // Another low address
    32'h4000_0000   // Mid-range address
  };

  logic [31:0] test_data [5] = '{
    32'hDEAD_BEEF,
    32'hCAFE_BABE,
    32'h1234_5678,
    32'hABCD_EF00,
    32'h5555_AAAA
  };

  // Unwritten test addresses
  logic [31:0] unwritten_addresses [3] = '{
    32'h0000_0500,  // Between written addresses
    32'h9000_0000,  // High unwritten address
    32'h0000_0000   // Very low address
  };

  initial begin
    // Setup waveform dumping
    $dumpfile("sparse_memory_testbench.vcd");
    $dumpvars(0, sparse_memory_testbench);

    $display("=== Sparse Memory Model Demonstration ===");
    $display();

    // Initialize control signals
    memory_controller_instance.write_enable = 1'b0;
    memory_controller_instance.read_enable = 1'b0;
    memory_controller_instance.memory_address = 32'h0000_0000;
    memory_controller_instance.write_data = 32'h0000_0000;

    #10;

    // Phase 1: Write data to sparse addresses
    $display("Phase 1: Writing to sparse memory addresses");
    for (int i = 0; i < 5; i++) begin
      #10;
      memory_controller_instance.memory_address = test_addresses[i];
      memory_controller_instance.write_data = test_data[i];
      memory_controller_instance.read_enable = 1'b0;
      #1;
      memory_controller_instance.write_enable = 1'b1;
      #1;
      memory_controller_instance.write_enable = 1'b0;
      #1;
    end

    #10;
    $display();
    memory_controller_instance.display_memory_statistics();
    $display();

    // Phase 2: Read from written addresses
    $display("Phase 2: Reading from written addresses");
    for (int i = 0; i < 5; i++) begin
      #10;
      memory_controller_instance.memory_address = test_addresses[i];
      memory_controller_instance.write_enable = 1'b0;
      memory_controller_instance.read_enable = 1'b1;
      #1;
      $display("READ:  Address 0x%08h = 0x%08h (VALID)", 
               test_addresses[i], memory_controller_instance.read_data);
      memory_controller_instance.read_enable = 1'b0;
      #1;
    end

    #10;
    $display();

    // Phase 3: Read from unwritten addresses
    $display("Phase 3: Reading from uninitialized addresses");

    for (int i = 0; i < 3; i++) begin
      #10;
      memory_controller_instance.memory_address = unwritten_addresses[i];
      memory_controller_instance.write_enable = 1'b0;
      memory_controller_instance.read_enable = 1'b1;
      #1;
      $display("READ:  Address 0x%08h = 0x%08h (UNINITIALIZED)", 
               unwritten_addresses[i], memory_controller_instance.read_data);
      memory_controller_instance.read_enable = 1'b0;
      #1;
    end

    #10;
    $display();
    $display("=== Test Complete ===");
    $display("Sparse memory model efficiently stores only 5 addresses");
    $display();

    #50;
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Sparse Memory Model Demonstration ===

Phase 1: Writing to sparse memory addresses
WRITE: Address 0x00001000 = 0xdeadbeef
WRITE: Address 0x80000000 = 0xcafebabe
WRITE: Address 0xfffff000 = 0x12345678
WRITE: Address 0x00002000 = 0xabcdef00
WRITE: Address 0x40000000 = 0x5555aaaa

Memory Statistics: 5 addresses in use
Stored addresses:
  0x00001000: 0xdeadbeef
  0x00002000: 0xabcdef00
  0x40000000: 0x5555aaaa
  0x80000000: 0xcafebabe
  0xfffff000: 0x12345678

Phase 2: Reading from written addresses
READ:  Address 0x00001000 = 0xdeadbeef (VALID)
READ:  Address 0x80000000 = 0xcafebabe (VALID)
READ:  Address 0xfffff000 = 0x12345678 (VALID)
READ:  Address 0x00002000 = 0xabcdef00 (VALID)
READ:  Address 0x40000000 = 0x5555aaaa (VALID)

Phase 3: Reading from uninitialized addresses
READ:  Address 0x00000500 = 0x5555aaaa (UNINITIALIZED)
READ:  Address 0x90000000 = 0x5555aaaa (UNINITIALIZED)
READ:  Address 0x00000000 = 0x5555aaaa (UNINITIALIZED)

=== Test Complete 

0

## Queues

Queues are ordered collections that allow insertion and deletion at both ends, making them perfect for FIFO and LIFO operations.

### Basic Queue Operations

```systemverilog
module queue_operations;
    int data_queue[$];
    
    initial begin
        // Push elements to back
        data_queue.push_back(10);
        data_queue.push_back(20);
        data_queue.push_back(30);
        
        // Push elements to front
        data_queue.push_front(5);
        
        $display("Queue after pushes: %p", data_queue);
        $display("Queue size: %0d", data_queue.size());
        
        // Pop elements
        int front_val = data_queue.pop_front();
        int back_val = data_queue.pop_back();
        
        $display("Popped front: %0d, back: %0d", front_val, back_val);
        $display("Queue after pops: %p", data_queue);
        
        // Insert at specific position
        data_queue.insert(1, 15);
        $display("After insert at index 1: %p", data_queue);
        
        // Delete from specific position
        data_queue.delete(0);
        $display("After delete index 0: %p", data_queue);
    end
endmodule
```

### Advanced Queue Methods

```systemverilog
module advanced_queue_methods;
    string cmd_queue[$];
    int numbers[$] = {1, 2, 3, 4, 5, 2, 6, 2, 7};
    
    initial begin
        // Queue manipulation
        cmd_queue = {"read", "write", "execute", "read", "delete"};
        
        $display("Original queue: %p", cmd_queue);
        
        // Find operations
        int index[$] = cmd_queue.find_index with (item == "read");
        $display("Indices with 'read': %p", index);
        
        string first_read[$] = cmd_queue.find_first with (item == "read");
        $display("First 'read' command: %p", first_read);
        
        // Working with numbers
        $display("Numbers: %p", numbers);
        
        // Find all occurrences of 2
        int twos[$] = numbers.find with (item == 2);
        $display("All 2's: %p", twos);
        
        // Find numbers greater than 4
        int big_nums[$] = numbers.find with (item > 4);
        $display("Numbers > 4: %p", big_nums);
        
        // Reverse the queue
        numbers.reverse();
        $display("Reversed: %p", numbers);
        
        // Sort the queue
        numbers.sort();
        $display("Sorted: %p", numbers);
        
        // Shuffle the queue
        numbers.shuffle();
        $display("Shuffled: %p", numbers);
    end
endmodule
```

### **Example 7: FIFO Buffer**
Simple first-in-first-out queue showing push_back() and pop_front() operations.

In [7]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_7__fifo_buffer/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// fifo_buffer_design.sv
module simple_fifo_buffer #(
  parameter DATA_WIDTH = 8,
  parameter FIFO_DEPTH = 4
)(
  input  logic                  clock_signal,
  input  logic                  reset_signal,
  input  logic                  push_enable,
  input  logic                  pop_enable,
  input  logic [DATA_WIDTH-1:0] data_input,
  output logic [DATA_WIDTH-1:0] data_output,
  output logic                  fifo_empty_flag,
  output logic                  fifo_full_flag
);

  // Internal FIFO storage array
  logic [DATA_WIDTH-1:0] fifo_memory [0:FIFO_DEPTH-1];
  
  // Pointers for head and tail of FIFO
  logic [$clog2(FIFO_DEPTH):0] write_pointer;
  logic [$clog2(FIFO_DEPTH):0] read_pointer;
  logic [$clog2(FIFO_DEPTH):0] data_count;

  // Status flags
  assign fifo_empty_flag = (data_count == 0);
  assign fifo_full_flag  = (data_count == FIFO_DEPTH);

  // Output data from read pointer location
  assign data_output = fifo_memory[read_pointer[1:0]];

  always_ff @(posedge clock_signal or posedge reset_signal) begin
    if (reset_signal) begin
      write_pointer <= 0;
      read_pointer  <= 0;
      data_count    <= 0;
    end
    else begin
      // Push operation (push_back)
      if (push_enable && !fifo_full_flag) begin
        fifo_memory[write_pointer[1:0]] <= data_input;
        write_pointer <= write_pointer + 1;
        data_count    <= data_count + 1;
        $display("FIFO: Pushed data 0x%02x, count=%0d", 
                 data_input, data_count + 1);
      end
      
      // Pop operation (pop_front)
      if (pop_enable && !fifo_empty_flag) begin
        read_pointer <= read_pointer + 1;
        data_count   <= data_count - 1;
        $display("FIFO: Popped data 0x%02x, count=%0d", 
                 data_output, data_count - 1);
      end
    end
  end

endmodule
```

```systemverilog
// fifo_buffer_design_testbench.sv
module fifo_buffer_testbench;

  // Testbench signals
  logic        clock_signal;
  logic        reset_signal;
  logic        push_enable;
  logic        pop_enable;
  logic [7:0]  data_input;
  logic [7:0]  data_output;
  logic        fifo_empty_flag;
  logic        fifo_full_flag;

  // Instantiate the FIFO buffer design under test
  simple_fifo_buffer #(
    .DATA_WIDTH(8),
    .FIFO_DEPTH(4)
  ) FIFO_BUFFER_INSTANCE (
    .clock_signal(clock_signal),
    .reset_signal(reset_signal),
    .push_enable(push_enable),
    .pop_enable(pop_enable),
    .data_input(data_input),
    .data_output(data_output),
    .fifo_empty_flag(fifo_empty_flag),
    .fifo_full_flag(fifo_full_flag)
  );

  // Clock generation
  always #5 clock_signal = ~clock_signal;

  initial begin
    // Dump waves for analysis
    $dumpfile("fifo_buffer_testbench.vcd");
    $dumpvars(0, fifo_buffer_testbench);

    // Initialize all signals
    clock_signal = 0;
    reset_signal = 1;
    push_enable  = 0;
    pop_enable   = 0;
    data_input   = 8'h00;

    $display();
    $display("=== FIFO Buffer Test Starting ===");
    
    // Release reset
    #10 reset_signal = 0;
    #10;

    // Test push_back operations (filling FIFO)
    $display("\n--- Testing push_back operations ---");
    push_data_to_fifo(8'hAA);
    push_data_to_fifo(8'hBB);
    push_data_to_fifo(8'hCC);
    push_data_to_fifo(8'hDD);
    
    // Try to push when full
    $display("\n--- Testing push when FIFO is full ---");
    push_data_to_fifo(8'hEE);

    // Test pop_front operations (emptying FIFO)
    $display("\n--- Testing pop_front operations ---");
    pop_data_from_fifo();
    pop_data_from_fifo();
    pop_data_from_fifo();
    pop_data_from_fifo();
    
    // Try to pop when empty
    $display("\n--- Testing pop when FIFO is empty ---");
    pop_data_from_fifo();

    // Test mixed push and pop operations
    $display("\n--- Testing mixed push/pop operations ---");
    push_data_to_fifo(8'h11);
    push_data_to_fifo(8'h22);
    pop_data_from_fifo();
    push_data_to_fifo(8'h33);
    pop_data_from_fifo();
    pop_data_from_fifo();

    $display();
    $display("=== FIFO Buffer Test Complete ===");
    $display();
    
    #20 $finish;
  end

  // Task for push_back operation
  task push_data_to_fifo(input [7:0] test_data);
    data_input  = test_data;
    push_enable = 1;
    #10;
    push_enable = 0;
    #10;
  endtask

  // Task for pop_front operation  
  task pop_data_from_fifo();
    pop_enable = 1;
    #10;
    pop_enable = 0;
    #10;
  endtask

endmodule
```

Verilator Simulation Output:

=== FIFO Buffer Test Starting ===

--- Testing push_back operations ---
FIFO: Pushed data 0xaa, count=1
FIFO: Pushed data 0xbb, count=2
FIFO: Pushed data 0xcc, count=3
FIFO: Pushed data 0xdd, count=4

--- Testing push when FIFO is full ---

--- Testing pop_front operations ---
FIFO: Popped data 0xaa, count=3
FIFO: Popped data 0xbb, count=2
FIFO: Popped data 0xcc, count=1
FIFO: Popped data 0xdd, count=0

--- Testing pop when FIFO is empty ---

--- Testing mixed push/pop operations ---
FIFO: Pushed data 0x11, count=1
FIFO: Pushed data 0x22, count=2
FIFO: Popped data 0x11, count=1
FIFO: Pushed data 0x33, count=2
FIFO: Popped data 0x22, count=1
FIFO: Popped data 0x33, count=0

=== FIFO Buffer Test Complete ===

Process finished with return code: 0
Removing Chapter_8_examples/example_7__fifo_buffer/obj_dir directory...
Chapter_8_examples/example_7__fifo_buffer/obj_dir removed successfully.


0

### **Example 8: Command Queue**
Queue storing text commands that can be added to either end and processed in order.

In [8]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_8__command_queue/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// command_queue_processor.sv
module command_queue_processor #(
    parameter QUEUE_DEPTH = 8,
    parameter CMD_WIDTH = 64  // 8 characters * 8 bits each
) (
    input  logic                    clock,
    input  logic                    reset_n,
    input  logic                    push_front_enable,
    input  logic                    push_back_enable,
    input  logic                    pop_enable,
    input  logic [CMD_WIDTH-1:0]    command_data_in,
    output logic [CMD_WIDTH-1:0]    command_data_out,
    output logic                    queue_empty,
    output logic                    queue_full
);

    // Internal queue storage
    logic [CMD_WIDTH-1:0] command_queue [0:QUEUE_DEPTH-1];
    logic [$clog2(QUEUE_DEPTH)-1:0] front_pointer;
    logic [$clog2(QUEUE_DEPTH)-1:0] back_pointer;
    logic [$clog2(QUEUE_DEPTH):0] queue_count;

    // Status signals
    assign queue_empty = (queue_count == 0);
    assign queue_full = (queue_count == QUEUE_DEPTH);
    assign command_data_out = queue_empty ? '0 : 
                             command_queue[front_pointer];

    always_ff @(posedge clock or negedge reset_n) begin
        if (!reset_n) begin
            front_pointer <= 0;
            back_pointer <= 0;
            queue_count <= 0;
            for (int i = 0; i < QUEUE_DEPTH; i++) begin
                command_queue[i] <= '0;
            end
        end else begin
            // Handle push to front
            if (push_front_enable && !queue_full) begin
                if (front_pointer == 0) begin
                    front_pointer <= $clog2(QUEUE_DEPTH)'(QUEUE_DEPTH - 1);
                end else begin
                    front_pointer <= front_pointer - 1;
                end
                command_queue[front_pointer == 0 ? QUEUE_DEPTH-1 : 
                             front_pointer-1] <= command_data_in;
                queue_count <= queue_count + 1;
            end
            // Handle push to back
            else if (push_back_enable && !queue_full) begin
                command_queue[back_pointer] <= command_data_in;
                if (back_pointer == $clog2(QUEUE_DEPTH)'(QUEUE_DEPTH - 1)) begin
                    back_pointer <= 0;
                end else begin
                    back_pointer <= back_pointer + 1;
                end
                queue_count <= queue_count + 1;
            end
            // Handle pop from front
            else if (pop_enable && !queue_empty) begin
                if (front_pointer == $clog2(QUEUE_DEPTH)'(QUEUE_DEPTH - 1)) begin
                    front_pointer <= 0;
                end else begin
                    front_pointer <= front_pointer + 1;
                end
                queue_count <= queue_count - 1;
            end
        end
    end

    // Display current command when it changes
    always @(command_data_out) begin
        if (!queue_empty && command_data_out != 0) begin
            $display("Processing command: %s", command_data_out);
        end
    end

endmodule
```

```systemverilog
// command_queue_processor_testbench.sv
module command_queue_testbench;

    // Testbench signals
    logic                   test_clock;
    logic                   test_reset_n;
    logic                   test_push_front_enable;
    logic                   test_push_back_enable;
    logic                   test_pop_enable;
    logic [63:0]           test_command_data_in;
    logic [63:0]           test_command_data_out;
    logic                   test_queue_empty;
    logic                   test_queue_full;

    // Instantiate design under test
    command_queue_processor COMMAND_QUEUE_INSTANCE (
        .clock(test_clock),
        .reset_n(test_reset_n),
        .push_front_enable(test_push_front_enable),
        .push_back_enable(test_push_back_enable),
        .pop_enable(test_pop_enable),
        .command_data_in(test_command_data_in),
        .command_data_out(test_command_data_out),
        .queue_empty(test_queue_empty),
        .queue_full(test_queue_full)
    );

    // Clock generation
    initial begin
        test_clock = 0;
        forever #5 test_clock = ~test_clock;  // 10ns period
    end

    // Test stimulus
    initial begin
        // Dump waves
        $dumpfile("command_queue_testbench.vcd");
        $dumpvars(0, command_queue_testbench);

        // Initialize signals
        test_reset_n = 0;
        test_push_front_enable = 0;
        test_push_back_enable = 0;
        test_pop_enable = 0;
        test_command_data_in = 0;

        $display("=== Command Queue Processor Test ===");
        $display();

        // Release reset
        #20 test_reset_n = 1;
        #10;

        // Test 1: Add commands to back of queue
        $display("Test 1: Adding commands to back of queue");
        add_command_to_back(64'h4D4F56450000_0000);  // "MOVE"
        add_command_to_back(64'h5475726E0000_0000);  // "Turn"
        add_command_to_back(64'h53746F7000000000);   // "Stop"
        #10;

        // Test 2: Process commands in order
        $display("Test 2: Processing commands in FIFO order");
        process_next_command();
        process_next_command();
        process_next_command();
        #10;

        // Test 3: Add urgent commands to front
        $display("Test 3: Adding urgent commands to front");
        add_command_to_back(64'h536C6F7700000000);   // "Slow"
        add_command_to_front(64'h4272616B65000000);  // "Brake"
        add_command_to_front(64'h555247454E540000);  // "URGENT"
        #10;

        // Test 4: Process mixed priority commands
        $display("Test 4: Processing mixed priority commands");
        process_next_command();  // Should be URGENT
        process_next_command();  // Should be Brake
        process_next_command();  // Should be Slow
        #10;

        // Test 5: Fill queue to capacity
        $display("Test 5: Testing queue capacity");
        repeat(8) begin
            add_command_to_back(64'h46554C4C00000000);  // "FULL"
        end
        $display("Queue full status: %b", test_queue_full);
        #10;

        // Test 6: Empty the queue
        $display("Test 6: Emptying the queue");
        repeat(8) begin
            process_next_command();
        end
        $display("Queue empty status: %b", test_queue_empty);
        #20;

        $display();
        $display("=== Command Queue Test Complete ===");
        $finish;
    end

    // Helper task to add command to back
    task add_command_to_back(input [63:0] cmd_data);
        begin
            test_command_data_in = cmd_data;
            test_push_back_enable = 1;
            @(posedge test_clock);
            test_push_back_enable = 0;
            test_command_data_in = 0;
            $display("Added command to back of queue");
        end
    endtask

    // Helper task to add urgent command to front
    task add_command_to_front(input [63:0] cmd_data);
        begin
            test_command_data_in = cmd_data;
            test_push_front_enable = 1;
            @(posedge test_clock);
            test_push_front_enable = 0;
            test_command_data_in = 0;
            $display("Added urgent command to front of queue");
        end
    endtask

    // Helper task to process next command
    task process_next_command();
        begin
            if (!test_queue_empty) begin
                test_pop_enable = 1;
                @(posedge test_clock);
                test_pop_enable = 0;
                $display("Processed and removed command from queue");
            end else begin
                $display("Cannot process: Queue is empty");
            end
        end
    endtask

endmodule
```

Verilator Simulation Output:
=== Command Queue Processor Test ===

Test 1: Adding commands to back of queue
Added command to back of queue
Processing command: Turn
Added command to back of queue
Added command to back of queue
Test 2: Processing commands in FIFO order
Processing command: Stop
Processed and removed command from queue
Processed and removed command from queue
Cannot process: Queue is empty
Test 3: Adding urgent commands to front
Processing command: Slow
Added command to back of queue
Processing command: Brake
Added urgent command to front of queue
Processing command: URGENT
Added urgent command to front of queue
Test 4: Processing mixed priority commands
Processing command: Brake
Processed and removed command from queue
Processing command: Slow
Processed and removed command from queue
Processed and removed command from queue
Test 5: Testing queue capacity
Processing command: FULL
Added command to back of queue
Added command to back of queue
Added command to back of queue
A

0

### **Example 9: Number Sorting**
Using queue methods to find, sort, and shuffle a collection of numbers.

In [9]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_9__number_sorting/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// number_sorting_processor.sv
module number_sorting_processor ();               // Design under test
  
  // Queue to hold our collection of numbers
  int number_collection[$];
  int sorted_numbers[$];
  int temp_value;
  int i, j;
  int collection_size;
  
  initial begin
    $display();                                    // Display empty line
    $display("Number Sorting Processor Started!");
    
    // Initialize collection with some numbers
    number_collection.push_back(45);
    number_collection.push_back(12);
    number_collection.push_back(78);
    number_collection.push_back(23);
    number_collection.push_back(56);
    number_collection.push_back(34);
    
    $display("Original collection size: %0d", number_collection.size());
    $display("Original numbers:");
    for (i = 0; i < number_collection.size(); i++) begin
      $display("  Position %0d: %0d", i, number_collection[i]);
    end
    
    // Find maximum number manually
    temp_value = number_collection[0];
    for (i = 1; i < number_collection.size(); i++) begin
      if (number_collection[i] > temp_value) begin
        temp_value = number_collection[i];
      end
    end
    $display("Maximum number found: %0d", temp_value);
    
    // Find minimum number manually
    temp_value = number_collection[0];
    for (i = 1; i < number_collection.size(); i++) begin
      if (number_collection[i] < temp_value) begin
        temp_value = number_collection[i];
      end
    end
    $display("Minimum number found: %0d", temp_value);
    
    // Copy to sorted array for bubble sort
    sorted_numbers = number_collection;
    collection_size = sorted_numbers.size();
    
    // Bubble sort implementation (ascending order)
    for (i = 0; i < collection_size - 1; i++) begin
      for (j = 0; j < collection_size - i - 1; j++) begin
        if (sorted_numbers[j] > sorted_numbers[j + 1]) begin
          // Swap elements
          temp_value = sorted_numbers[j];
          sorted_numbers[j] = sorted_numbers[j + 1];
          sorted_numbers[j + 1] = temp_value;
        end
      end
    end
    
    $display("Sorted ascending:");
    for (i = 0; i < sorted_numbers.size(); i++) begin
      $display("  Position %0d: %0d", i, sorted_numbers[i]);
    end
    
    // Reverse the sorted array for descending order
    for (i = 0; i < collection_size / 2; i++) begin
      temp_value = sorted_numbers[i];
      sorted_numbers[i] = sorted_numbers[collection_size - 1 - i];
      sorted_numbers[collection_size - 1 - i] = temp_value;
    end
    
    $display("Sorted descending:");
    for (i = 0; i < sorted_numbers.size(); i++) begin
      $display("  Position %0d: %0d", i, sorted_numbers[i]);
    end
    
    $display("Number Sorting Processing Complete!");
    $display();                                    // Display empty line
  end
  
endmodule
```

```systemverilog
// number_sorting_processor_testbench.sv
module number_sorting_testbench;              // Testbench module
  
  // Instantiate design under test
  number_sorting_processor SORTING_PROCESSOR_INSTANCE();  
  
  // Additional testbench variables for testing
  int test_numbers[$];
  int found_count;
  int search_value;
  int sum_result;
  int temp_value;
  int i;
  
  initial begin
    // Dump waves
    $dumpfile("number_sorting_testbench.vcd");     // Specify VCD file
    $dumpvars(0, number_sorting_testbench);        // Dump all variables
    
    #1;                                            // Wait for time unit
    
    $display("Hello from number sorting testbench!");
    $display();                                    // Display empty line
    
    // Test additional queue operations
    test_numbers.push_back(100);
    test_numbers.push_back(25);
    test_numbers.push_back(75);
    test_numbers.push_back(50);
    test_numbers.push_back(75);  // Duplicate for testing
    
    $display("Test collection:");
    for (i = 0; i < test_numbers.size(); i++) begin
      $display("  Element %0d: %0d", i, test_numbers[i]);
    end
    
    // Test finding specific values
    search_value = 75;
    found_count = 0;
    $display("Searching for value: %0d", search_value);
    for (i = 0; i < test_numbers.size(); i++) begin
      if (test_numbers[i] == search_value) begin
        $display("  Found %0d at position %0d", search_value, i);
        found_count++;
      end
    end
    $display("Total occurrences of %0d: %0d", search_value, found_count);
    
    // Test finding numbers greater than threshold
    search_value = 50;
    found_count = 0;
    $display("Numbers greater than %0d:", search_value);
    for (i = 0; i < test_numbers.size(); i++) begin
      if (test_numbers[i] > search_value) begin
        $display("  Found: %0d", test_numbers[i]);
        found_count++;
      end
    end
    $display("Count of numbers > %0d: %0d", search_value, found_count);
    
    // Test sum calculation
    sum_result = 0;
    for (i = 0; i < test_numbers.size(); i++) begin
      sum_result += test_numbers[i];
    end
    $display("Sum of all numbers: %0d", sum_result);
    
    // Test queue operations: pop and push
    $display("Queue size before pop: %0d", test_numbers.size());
    if (test_numbers.size() > 0) begin
      temp_value = test_numbers.pop_back();
      $display("Popped value: %0d", temp_value);
      $display("Queue size after pop: %0d", test_numbers.size());
    end
    
    // Add new element
    test_numbers.push_front(999);
    $display("Added 999 to front, new size: %0d", test_numbers.size());
    $display("First element is now: %0d", test_numbers[0]);
    
    $display();                                    // Display empty line
    $display("Testbench verification completed!");
    
  end
  
endmodule
```

Verilator Simulation Output:

Number Sorting Processor Started!
Original collection size: 6
Original numbers:
  Position 0: 45
  Position 1: 12
  Position 2: 78
  Position 3: 23
  Position 4: 56
  Position 5: 34
Maximum number found: 78
Minimum number found: 12
Sorted ascending:
  Position 0: 12
  Position 1: 23
  Position 2: 34
  Position 3: 45
  Position 4: 56
  Position 5: 78
Sorted descending:
  Position 0: 78
  Position 1: 56
  Position 2: 45
  Position 3: 34
  Position 4: 23
  Position 5: 12
Number Sorting Processing Complete!

Hello from number sorting testbench!

Test collection:
  Element 0: 100
  Element 1: 25
  Element 2: 75
  Element 3: 50
  Element 4: 75
Searching for value: 75
  Found 75 at position 2
  Found 75 at position 4
Total occurrences of 75: 2
Numbers greater than 50:
  Found: 100
  Found: 75
  Found: 75
Count of numbers > 50: 3
Sum of all numbers: 325
Queue size before pop: 5
Popped value: 75
Queue size after pop: 4
Added 999 to front, new size: 5
First element 

0

## Packed Arrays and Structures

Packed arrays and structures allow you to group related data while maintaining bit-level access and efficient storage.

### Packed Arrays

```systemverilog
module packed_arrays;
    // Packed array declarations
    bit [7:0][3:0] nibble_array;  // 8 nibbles (4-bit each)
    logic [3:0][7:0] byte_array;  // 4 bytes
    
    initial begin
        // Initialize packed array
        nibble_array = 32'hABCD_EF01;
        
        $display("Full array: 0x%08h", nibble_array);
        
        // Access individual elements
        for (int i = 0; i < 8; i++) begin
            $display("nibble_array[%0d] = 0x%h", i, nibble_array[i]);
        end
        
        // Bit slicing
        $display("Upper 16 bits: 0x%04h", nibble_array[7:4]);
        $display("Lower 16 bits: 0x%04h", nibble_array[3:0]);
        
        // Byte-level access
        byte_array = 32'h12345678;
        $display("byte_array = 0x%08h", byte_array);
        
        foreach(byte_array[i]) begin
            $display("byte_array[%0d] = 0x%02h", i, byte_array[i]);
        end
    end
endmodule
```

### Packed Structures

```systemverilog
// Packed structure for CPU instruction
typedef struct packed {
    logic [5:0]  opcode;
    logic [4:0]  rs;
    logic [4:0]  rt;
    logic [15:0] immediate;
} cpu_instruction_t;

// Packed union for different data interpretations
typedef union packed {
    logic [31:0] word;
    logic [15:0] halfword [1:0];
    logic [7:0]  byte [3:0];
    cpu_instruction_t instruction;
} data_union_t;

module packed_structures;
    cpu_instruction_t instr;
    data_union_t data;
    
    initial begin
        // Create instruction
        instr.opcode = 6'b100011;    // Load word
        instr.rs = 5'd1;             // Base register
        instr.rt = 5'd2;             // Target register
        instr.immediate = 16'h0100;  // Offset
        
        $display("Instruction fields:");
        $display("  Opcode: 0b%06b", instr.opcode);
        $display("  RS: %0d", instr.rs);
        $display("  RT: %0d", instr.rt);
        $display("  Immediate: 0x%04h", instr.immediate);
        $display("  Full instruction: 0x%08h", instr);
        
        // Use union for different interpretations
        data.word = 32'hDEADBEEF;
        
        $display("\nUnion interpretations:");
        $display("  Word: 0x%08h", data.word);
        $display("  Halfwords: 0x%04h 0x%04h", 
                 data.halfword[1], data.halfword[0]);
        $display("  Bytes: 0x%02h 0x%02h 0x%02h 0x%02h",
                 data.byte[3], data.byte[2], data.byte[1], data.byte[0]);
        
        // Interpret as instruction
        $display("  As instruction - Opcode: 0b%06b", data.instruction.opcode);
    end
endmodule
```

### **Example 10: Byte Array Operations**
Working with packed arrays as collections of bytes with bit-level access.

In [10]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_10__byte_array_operations/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// byte_array_processor.sv
module byte_array_processor (
  input  logic        clock,
  input  logic        reset,
  input  logic [31:0] data_word,           // 4-byte input word
  input  logic [1:0]  byte_select,         // Select which byte to access
  input  logic        write_enable,        // Enable write operation
  output logic [7:0]  selected_byte,       // Output selected byte
  output logic        byte_parity,         // Parity of selected byte
  output logic [3:0]  nibble_high,         // Upper 4 bits of byte
  output logic [3:0]  nibble_low           // Lower 4 bits of byte
);

  // Internal byte array storage (4 bytes = 32 bits)
  logic [31:0] byte_array_storage;

  // Byte extraction from packed array
  logic [7:0] byte_0, byte_1, byte_2, byte_3;
  
  always_comb begin
    // Extract individual bytes from packed array
    byte_0 = byte_array_storage[7:0];       // LSB byte
    byte_1 = byte_array_storage[15:8];      // Byte 1
    byte_2 = byte_array_storage[23:16];     // Byte 2  
    byte_3 = byte_array_storage[31:24];     // MSB byte
    
    // Select byte based on byte_select input
    case (byte_select)
      2'b00: selected_byte = byte_0;
      2'b01: selected_byte = byte_1;
      2'b10: selected_byte = byte_2;
      2'b11: selected_byte = byte_3;
    endcase
    
    // Bit-level operations on selected byte
    byte_parity = ^selected_byte;           // XOR reduction for parity
    nibble_high = selected_byte[7:4];       // Upper nibble
    nibble_low  = selected_byte[3:0];       // Lower nibble
  end

  // Sequential logic for storage updates
  always_ff @(posedge clock or posedge reset) begin
    if (reset) begin
      byte_array_storage <= 32'h00000000;
    end else if (write_enable) begin
      byte_array_storage <= data_word;
    end
  end

  // Display byte array contents for debugging
  initial begin
    $display("Byte Array Processor initialized");
  end

endmodule
```

```systemverilog
// byte_array_processor_testbench.sv
module byte_array_testbench;

  // Testbench signals
  logic        test_clock;
  logic        test_reset;
  logic [31:0] test_data_word;
  logic [1:0]  test_byte_select;
  logic        test_write_enable;
  logic [7:0]  result_selected_byte;
  logic        result_byte_parity;
  logic [3:0]  result_nibble_high;
  logic [3:0]  result_nibble_low;

  // Instantiate design under test
  byte_array_processor BYTE_PROCESSOR_INSTANCE (
    .clock(test_clock),
    .reset(test_reset),
    .data_word(test_data_word),
    .byte_select(test_byte_select),
    .write_enable(test_write_enable),
    .selected_byte(result_selected_byte),
    .byte_parity(result_byte_parity),
    .nibble_high(result_nibble_high),
    .nibble_low(result_nibble_low)
  );

  // Clock generation
  always #5 test_clock = ~test_clock;

  initial begin
    // Dump waves
    $dumpfile("byte_array_testbench.vcd");
    $dumpvars(0, byte_array_testbench);
    
    // Initialize signals
    test_clock = 0;
    test_reset = 1;
    test_data_word = 32'h00000000;
    test_byte_select = 2'b00;
    test_write_enable = 0;
    
    $display("=== Byte Array Operations Test ===");
    $display();
    
    // Release reset
    #10 test_reset = 0;
    
    // Test 1: Write test pattern and access different bytes
    $display("Test 1: Writing byte pattern 0xDEADBEEF");
    test_data_word = 32'hDEADBEEF;
    test_write_enable = 1;
    #10;
    test_write_enable = 0;
    
    // Access each byte and show bit-level operations
    for (int byte_index = 0; byte_index < 4; byte_index++) begin
      test_byte_select = byte_index[1:0];
      #5;
      $display("Byte[%0d]: 0x%02h, Parity: %b, High: 0x%01h, Low: 0x%01h",
               byte_index, result_selected_byte, result_byte_parity,
               result_nibble_high, result_nibble_low);
    end
    
    $display();
    
    // Test 2: Write different pattern and test bit access
    $display("Test 2: Writing byte pattern 0x12345678");
    test_data_word = 32'h12345678;
    test_write_enable = 1;
    #10;
    test_write_enable = 0;
    
    // Show byte-level access with bit operations
    for (int byte_index = 0; byte_index < 4; byte_index++) begin
      test_byte_select = byte_index[1:0];
      #5;
      $display("Byte[%0d]: 0x%02h = 8'b%08b, Nibbles: %04b_%04b",
               byte_index, result_selected_byte, result_selected_byte,
               result_nibble_high, result_nibble_low);
    end
    
    $display();
    $display("Byte array operations test completed!");
    
    #10 $finish;
  end

endmodule
```

Verilator Simulation Output:
Byte Array Processor initialized
=== Byte Array Operations Test ===

Test 1: Writing byte pattern 0xDEADBEEF
Byte[0]: 0xef, Parity: 1, High: 0xe, Low: 0xf
Byte[1]: 0xbe, Parity: 0, High: 0xb, Low: 0xe
Byte[2]: 0xad, Parity: 1, High: 0xa, Low: 0xd
Byte[3]: 0xde, Parity: 0, High: 0xd, Low: 0xe

Test 2: Writing byte pattern 0x12345678
Byte[0]: 0x78 = 8'b01111000, Nibbles: 0111_1000
Byte[1]: 0x56 = 8'b01010110, Nibbles: 0101_0110
Byte[2]: 0x34 = 8'b00110100, Nibbles: 0011_0100
Byte[3]: 0x12 = 8'b00010010, Nibbles: 0001_0010

Byte array operations test completed!
Process finished with return code: 0
Removing Chapter_8_examples/example_10__byte_array_operations/obj_dir directory...
Chapter_8_examples/example_10__byte_array_operations/obj_dir removed successfully.


0

### **Example 11: CPU Instruction Format**
Packed structure representing a simple CPU instruction with opcode and operand fields.

In [11]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_11__cpu_instruction_format/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// cpu_instruction_decoder.sv
module cpu_instruction_decoder ();

  // Define packed structure for CPU instruction format (16-bit instruction)
  typedef struct packed {
    logic [3:0]  opcode;        // 4-bit operation code
    logic [7:0]  operand_addr;  // 8-bit operand address
    logic [3:0]  register_sel;  // 4-bit register selection
  } cpu_instruction_format_t;

  // Define opcode constants for readability
  typedef enum logic [3:0] {
    OP_NOP  = 4'b0000,  // No operation
    OP_LOAD = 4'b0001,  // Load from memory
    OP_STORE= 4'b0010,  // Store to memory  
    OP_ADD  = 4'b0011,  // Addition
    OP_SUB  = 4'b0100,  // Subtraction
    OP_JUMP = 4'b0101,  // Jump instruction
    OP_HALT = 4'b1111   // Halt processor
  } opcode_enum_t;

  // Create instruction instances
  cpu_instruction_format_t current_instruction;
  cpu_instruction_format_t sample_instructions[5];

  initial begin
    $display("=== CPU Instruction Format Demonstration ===");
    $display();

    // Initialize sample instructions
    sample_instructions[0] = {OP_LOAD,  8'h42, 4'h1};  // Load from addr 0x42
    sample_instructions[1] = {OP_ADD,   8'h00, 4'h2};  // Add to register 2
    sample_instructions[2] = {OP_STORE, 8'h50, 4'h2};  // Store reg 2 to 0x50
    sample_instructions[3] = {OP_JUMP,  8'h10, 4'h0};  // Jump to address 0x10
    sample_instructions[4] = {OP_HALT,  8'h00, 4'h0};  // Halt processor

    // Display instruction format information
    $display("Instruction Format (16-bit total):");
    $display("  [15:12] Opcode     - 4 bits");
    $display("  [11:4]  Operand    - 8 bits"); 
    $display("  [3:0]   Register   - 4 bits");
    $display();

    // Process and display each instruction
    for (int instruction_index = 0; instruction_index < 5; 
         instruction_index++) begin
      current_instruction = sample_instructions[instruction_index];
      
      $display("Instruction %0d: 0x%04X", instruction_index,
               current_instruction);
      $display("  Opcode: %s (0x%01X)", 
               get_opcode_name(current_instruction.opcode),
               current_instruction.opcode);
      $display("  Operand Address: 0x%02X", 
               current_instruction.operand_addr);
      $display("  Register Select: R%0d", 
               current_instruction.register_sel);
      $display();
    end

    $display("=== End of CPU Instruction Demo ===");
  end

  // Function to convert opcode to readable string
  function string get_opcode_name(logic [3:0] opcode_value);
    case (opcode_value)
      OP_NOP:   return "NOP";
      OP_LOAD:  return "LOAD";
      OP_STORE: return "STORE";
      OP_ADD:   return "ADD";
      OP_SUB:   return "SUB";
      OP_JUMP:  return "JUMP";
      OP_HALT:  return "HALT";
      default:  return "UNKNOWN";
    endcase
  endfunction

endmodule
```

```systemverilog
// cpu_instruction_decoder_testbench.sv
module cpu_instruction_testbench;

  // Instantiate the CPU instruction decoder design
  cpu_instruction_decoder CPU_DECODER_INSTANCE();

  // Additional testbench-specific instruction testing
  typedef struct packed {
    logic [3:0]  opcode;        // 4-bit operation code
    logic [7:0]  operand_addr;  // 8-bit operand address  
    logic [3:0]  register_sel;  // 4-bit register selection
  } instruction_packet_t;

  instruction_packet_t test_instruction_sequence[3];
  logic [15:0] raw_instruction_bits;

  initial begin
    // Configure wave dump
    $dumpfile("cpu_instruction_testbench.vcd");
    $dumpvars(0, cpu_instruction_testbench);
    
    #1; // Wait for design to complete
    
    $display("=== Testbench: Additional Instruction Tests ===");
    $display();

    // Create test sequence with edge cases
    test_instruction_sequence[0] = {4'b0000, 8'hFF, 4'hF}; // All max values
    test_instruction_sequence[1] = {4'b1010, 8'h55, 4'h5}; // Pattern test
    test_instruction_sequence[2] = {4'b0101, 8'hAA, 4'hA}; // Alternating bits

    // Test instruction bit manipulation
    for (int test_index = 0; test_index < 3; test_index++) begin
      raw_instruction_bits = test_instruction_sequence[test_index];
      
      $display("Test %0d - Raw bits: 0b%016b", test_index, 
               raw_instruction_bits);
      $display("  Extracted Opcode: 0x%01X", 
               raw_instruction_bits[15:12]);
      $display("  Extracted Operand: 0x%02X", 
               raw_instruction_bits[11:4]);  
      $display("  Extracted Register: 0x%01X", 
               raw_instruction_bits[3:0]);
      $display();
    end

    // Demonstrate bit field access vs full word access
    test_instruction_sequence[0] = {4'h3, 8'h7F, 4'h8};
    $display("Field Access Demo:");
    $display("  Full instruction: 0x%04X", test_instruction_sequence[0]);
    $display("  Opcode field: %0d", test_instruction_sequence[0].opcode);
    $display("  Address field: %0d", 
             test_instruction_sequence[0].operand_addr);
    $display("  Register field: %0d", 
             test_instruction_sequence[0].register_sel);
    $display();

    $display("=== Testbench Complete ===");
    $display();
  end

endmodule
```

Verilator Simulation Output:
=== CPU Instruction Format Demonstration ===

Instruction Format (16-bit total):
  [15:12] Opcode     - 4 bits
  [11:4]  Operand    - 8 bits
  [3:0]   Register   - 4 bits

Instruction 0: 0x1421
  Opcode: LOAD (0x1)
  Operand Address: 0x42
  Register Select: R1

Instruction 1: 0x3002
  Opcode: ADD (0x3)
  Operand Address: 0x00
  Register Select: R2

Instruction 2: 0x2502
  Opcode: STORE (0x2)
  Operand Address: 0x50
  Register Select: R2

Instruction 3: 0x5100
  Opcode: JUMP (0x5)
  Operand Address: 0x10
  Register Select: R0

Instruction 4: 0xf000
  Opcode: HALT (0xf)
  Operand Address: 0x00
  Register Select: R0

=== End of CPU Instruction Demo ===
=== Testbench: Additional Instruction Tests ===

Test 0 - Raw bits: 0b0000111111111111
  Extracted Opcode: 0x0
  Extracted Operand: 0xff
  Extracted Register: 0xf

Test 1 - Raw bits: 0b1010010101010101
  Extracted Opcode: 0xa
  Extracted Operand: 0x55
  Extracted Register: 0x5

Test 2 - Raw bits: 0b0101101010101

0

### **Example 12: Register Overlay**
Using packed unions to view the same data as different formats (word, bytes, bits).

In [12]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_12__register_overlay/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// register_overlay_design.sv
module register_overlay_processor ();

  // Packed union to overlay different views of 32-bit data
  typedef union packed {
    logic [31:0] word_view;           // Full 32-bit word
    logic [3:0][7:0] byte_view;       // 4 bytes (MSB to LSB)
    logic [31:0] bit_view;            // Individual bits
    struct packed {
      logic [15:0] upper_half;        // Upper 16 bits
      logic [15:0] lower_half;        // Lower 16 bits  
    } half_word_view;
  } data_overlay_t;

  data_overlay_t control_register;
  data_overlay_t status_register;

  initial begin
    $display();
    $display("=== Register Overlay Demonstration ===");
    
    // Initialize control register with word view
    control_register.word_view = 32'hDEADBEEF;
    
    $display("Control Register Word: 0x%08X", 
             control_register.word_view);
    $display("  Byte[3]: 0x%02X", control_register.byte_view[3]);
    $display("  Byte[2]: 0x%02X", control_register.byte_view[2]); 
    $display("  Byte[1]: 0x%02X", control_register.byte_view[1]);
    $display("  Byte[0]: 0x%02X", control_register.byte_view[0]);
    $display("  Upper Half: 0x%04X", 
             control_register.half_word_view.upper_half);
    $display("  Lower Half: 0x%04X", 
             control_register.half_word_view.lower_half);
    
    // Modify individual bytes and observe word change
    status_register.byte_view[3] = 8'hAB;
    status_register.byte_view[2] = 8'hCD;
    status_register.byte_view[1] = 8'hEF;
    status_register.byte_view[0] = 8'h12;
    
    $display();
    $display("Status Register built from bytes:");
    $display("  Resulting Word: 0x%08X", status_register.word_view);
    
    // Modify half-words
    status_register.half_word_view.upper_half = 16'h5555;
    status_register.half_word_view.lower_half = 16'hAAAA;
    
    $display("  After half-word modification: 0x%08X", 
             status_register.word_view);
    $display();
  end

endmodule
```

```systemverilog
// register_overlay_design_testbench.sv
module register_overlay_testbench;

  // Instantiate design under test
  register_overlay_processor OVERLAY_PROCESSOR_INSTANCE();

  // Additional testbench-specific union for testing
  typedef union packed {
    logic [31:0] complete_word;
    logic [1:0][15:0] word_halves;
    logic [3:0][7:0] individual_bytes;
    struct packed {
      logic [7:0] flags;
      logic [7:0] control_bits;
      logic [15:0] data_payload;
    } register_fields;
  } test_register_t;

  test_register_t configuration_register;

  initial begin
    // Setup waveform dumping
    $dumpfile("register_overlay_testbench.vcd");
    $dumpvars(0, register_overlay_testbench);
    
    #1; // Wait for design initialization
    
    $display("=== Testbench Register Overlay Tests ===");
    
    // Test 1: Build register from individual components
    configuration_register.register_fields.flags = 8'b11010011;
    configuration_register.register_fields.control_bits = 8'b01010101;
    configuration_register.register_fields.data_payload = 16'h1234;
    
    $display("Test 1 - Building from fields:");
    $display("  Flags: 0b%08b", 
             configuration_register.register_fields.flags);
    $display("  Control: 0b%08b", 
             configuration_register.register_fields.control_bits);
    $display("  Payload: 0x%04X", 
             configuration_register.register_fields.data_payload);
    $display("  Complete Word: 0x%08X", 
             configuration_register.complete_word);
    
    #5;
    
    // Test 2: Modify bytes and check word
    configuration_register.individual_bytes[3] = 8'hFF;
    configuration_register.individual_bytes[2] = 8'h00;
    configuration_register.individual_bytes[1] = 8'hAA;
    configuration_register.individual_bytes[0] = 8'h55;
    
    $display();
    $display("Test 2 - Byte-level modifications:");
    $display("  Word after byte changes: 0x%08X", 
             configuration_register.complete_word);
    $display("  Upper half: 0x%04X", 
             configuration_register.word_halves[1]);
    $display("  Lower half: 0x%04X", 
             configuration_register.word_halves[0]);
    
    #5;
    
    // Test 3: Word-level assignment, check individual parts
    configuration_register.complete_word = 32'h87654321;
    
    $display();
    $display("Test 3 - Word assignment breakdown:");
    $display("  Full word: 0x%08X", 
             configuration_register.complete_word);
    $display("  Flags field: 0x%02X", 
             configuration_register.register_fields.flags);
    $display("  Control field: 0x%02X", 
             configuration_register.register_fields.control_bits);
    $display("  Data field: 0x%04X", 
             configuration_register.register_fields.data_payload);
    
    $display();
    $display("Hello from register overlay testbench!");
    $display();

  end

endmodule
```

Verilator Simulation Output:

=== Register Overlay Demonstration ===
Control Register Word: 0xdeadbeef
  Byte[3]: 0xde
  Byte[2]: 0xad
  Byte[1]: 0xbe
  Byte[0]: 0xef
  Upper Half: 0xdead
  Lower Half: 0xbeef

Status Register built from bytes:
  Resulting Word: 0xabcdef12
  After half-word modification: 0x5555aaaa

=== Testbench Register Overlay Tests ===
Test 1 - Building from fields:
  Flags: 0b11010011
  Control: 0b01010101
  Payload: 0x1234
  Complete Word: 0xd3551234

Test 2 - Byte-level modifications:
  Word after byte changes: 0xff00aa55
  Upper half: 0xff00
  Lower half: 0xaa55

Test 3 - Word assignment breakdown:
  Full word: 0x87654321
  Flags field: 0x87
  Control field: 0x65
  Data field: 0x4321

Hello from register overlay testbench!
Process finished with return code: 0
Removing Chapter_8_examples/example_12__register_overlay/obj_dir directory...
Chapter_8_examples/example_12__register_overlay/obj_dir removed successfully.


0

## Typedef Declarations

The `typedef` keyword allows you to create aliases for existing types, making code more readable and maintainable.

### Basic Typedef Usage

```systemverilog
// Basic type aliases
typedef logic [31:0] word_t;
typedef logic [15:0] halfword_t;
typedef logic [7:0]  byte_t;

// Array type aliases
typedef int int_array_t[10];
typedef real real_queue_t[$];
typedef string str_assoc_t[int];

module typedef_examples;
    word_t address, data;
    halfword_t port_id;
    byte_t status_reg;
    
    int_array_t fixed_buffer;
    real_queue_t floating_values;
    str_assoc_t error_messages;
    
    initial begin
        // Use typedef'd types
        address = 32'hDEADBEEF;
        data = 32'h12345678;
        port_id = 16'hABCD;
        status_reg = 8'h55;
        
        $display("Address: 0x%h", address);
        $display("Data: 0x%h", data);
        $display("Port ID: 0x%h", port_id);
        $display("Status: 0x%h", status_reg);
        
        // Initialize arrays
        foreach(fixed_buffer[i]) fixed_buffer[i] = i * i;
        
        floating_values.push_back(3.14159);
        floating_values.push_back(2.71828);
        
        error_messages[404] = "Not Found";
        error_messages[500] = "Internal Server Error";
        
        $display("Fixed buffer[5] = %0d", fixed_buffer[5]);
        $display("Floating values: %p", floating_values);
        $display("Error 404: %s", error_messages[404]);
    end
endmodule
```

### Complex Typedef Examples

```systemverilog
// Function pointer typedef
typedef function int math_func_t(int a, int b);

// Class handle typedef
typedef class transaction_c;
typedef transaction_c transaction_handle_t;

// Structure typedef with parameters
typedef struct {
    logic [DATA_WIDTH-1:0] data;
    logic valid;
    logic ready;
} handshake_if_t;

// Parameterized typedef
parameter int DATA_WIDTH = 32;
typedef handshake_if_t #(.DATA_WIDTH(DATA_WIDTH)) bus_if_t;

module complex_typedef;
    // Function implementations
    function int add_func(int a, int b);
        return a + b;
    endfunction
    
    function int mul_func(int a, int b);
        return a * b;
    endfunction
    
    // Function pointer usage
    math_func_t operation;
    
    initial begin
        int result;
        
        // Assign function to pointer
        operation = add_func;
        result = operation(5, 3);
        $display("Addition result: %0d", result);
        
        operation = mul_func;
        result = operation(5, 3);
        $display("Multiplication result: %0d", result);
    end
endmodule
```

### **Example 13: Custom Data Types**
Creating type aliases for common data widths (byte_t, word_t) to make code more readable.

In [13]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_13__custom_data_types/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// data_processor_design.sv
module data_processor_module (
  input  logic        clock_signal,
  input  logic        reset_signal,
  input  logic [7:0]  input_byte_data,
  input  logic [15:0] input_word_data,
  input  logic [31:0] input_dword_data,
  output logic [7:0]  output_byte_result,
  output logic [15:0] output_word_result,
  output logic [31:0] output_dword_result
);

  // Custom data type definitions for better code readability
  typedef logic [7:0]  byte_t;      // 8-bit byte type
  typedef logic [15:0] word_t;      // 16-bit word type  
  typedef logic [31:0] dword_t;     // 32-bit double word type
  typedef logic [63:0] qword_t;     // 64-bit quad word type

  // Internal registers using custom data types
  byte_t  internal_byte_register;
  word_t  internal_word_register;
  dword_t internal_dword_register;
  qword_t internal_qword_register;

  // Processing logic using custom data types
  always_ff @(posedge clock_signal or posedge reset_signal) begin
    if (reset_signal) begin
      internal_byte_register  <= byte_t'(0);
      internal_word_register  <= word_t'(0);
      internal_dword_register <= dword_t'(0);
      internal_qword_register <= qword_t'(0);
    end else begin
      // Process byte data - simple increment with proper width handling
      internal_byte_register <= byte_t'(byte_t'(input_byte_data) + 
                                        byte_t'(1));
      
      // Process word data - shift left by 1 with proper width handling
      internal_word_register <= word_t'(word_t'(input_word_data) << 1);
      
      // Process dword data - bitwise inversion with proper width handling
      internal_dword_register <= dword_t'(~dword_t'(input_dword_data));
      
      // Combine all inputs into qword register
      internal_qword_register <= qword_t'({dword_t'(input_dword_data), 
                                          word_t'(input_word_data), 
                                          byte_t'(input_byte_data),
                                          8'h00});
    end
  end

  // Output assignments using custom data types
  assign output_byte_result  = internal_byte_register;
  assign output_word_result  = internal_word_register;
  assign output_dword_result = internal_dword_register;

  // Display formatted output using custom types
  always_ff @(posedge clock_signal) begin
    if (!reset_signal) begin
      $display("Byte processing:  %02h -> %02h", 
               input_byte_data, internal_byte_register);
      $display("Word processing:  %04h -> %04h", 
               input_word_data, internal_word_register);
      $display("DWord processing: %08h -> %08h", 
               input_dword_data, internal_dword_register);
      $display("QWord combined:   %016h", internal_qword_register);
      $display("----------------------------------------");
    end
  end

endmodule
```

```systemverilog
// data_processor_design_testbench.sv
module data_processor_testbench;

  // Custom data type definitions (same as design for consistency)
  typedef logic [7:0]  byte_t;      // 8-bit byte type
  typedef logic [15:0] word_t;      // 16-bit word type  
  typedef logic [31:0] dword_t;     // 32-bit double word type

  // Testbench signals using custom data types
  logic   clock_signal;
  logic   reset_signal;
  byte_t  stimulus_byte_data;
  word_t  stimulus_word_data;
  dword_t stimulus_dword_data;
  byte_t  captured_byte_result;
  word_t  captured_word_result;
  dword_t captured_dword_result;

  // Clock generation
  initial begin
    clock_signal = 0;
    forever #5 clock_signal = ~clock_signal;  // 10ns period clock
  end

  // Design under test instantiation
  data_processor_module PROCESSOR_INSTANCE (
    .clock_signal       (clock_signal),
    .reset_signal       (reset_signal),
    .input_byte_data    (stimulus_byte_data),
    .input_word_data    (stimulus_word_data),
    .input_dword_data   (stimulus_dword_data),
    .output_byte_result (captured_byte_result),
    .output_word_result (captured_word_result),
    .output_dword_result(captured_dword_result)
  );

  // Test stimulus and verification
  initial begin
    // Initialize waveform dumping
    $dumpfile("data_processor_testbench.vcd");
    $dumpvars(0, data_processor_testbench);

    $display("=== Custom Data Types Example ===");
    $display("Testing byte_t, word_t, and dword_t definitions");
    $display();

    // Initialize signals
    reset_signal = 1;
    stimulus_byte_data  = byte_t'(8'h00);
    stimulus_word_data  = word_t'(16'h0000);
    stimulus_dword_data = dword_t'(32'h00000000);

    // Release reset after 2 clock cycles
    repeat(2) @(posedge clock_signal);
    reset_signal = 0;
    $display("Reset released - starting data processing tests");
    $display();

    // Test case 1: Small values
    @(posedge clock_signal);
    stimulus_byte_data  = byte_t'(8'h0F);
    stimulus_word_data  = word_t'(16'h1234);
    stimulus_dword_data = dword_t'(32'hABCD_EF01);
    
    @(posedge clock_signal);
    @(posedge clock_signal);

    // Test case 2: Maximum values
    @(posedge clock_signal);
    stimulus_byte_data  = byte_t'(8'hFF);
    stimulus_word_data  = word_t'(16'hFFFF);
    stimulus_dword_data = dword_t'(32'hFFFF_FFFF);
    
    @(posedge clock_signal);
    @(posedge clock_signal);

    // Test case 3: Pattern testing
    @(posedge clock_signal);
    stimulus_byte_data  = byte_t'(8'h55);    // Alternating pattern
    stimulus_word_data  = word_t'(16'h5AA5);  // Alternating pattern
    stimulus_dword_data = dword_t'(32'h5555_AAAA);
    
    @(posedge clock_signal);
    @(posedge clock_signal);

    // Test case 4: Zero values
    @(posedge clock_signal);
    stimulus_byte_data  = byte_t'(8'h00);
    stimulus_word_data  = word_t'(16'h0000);
    stimulus_dword_data = dword_t'(32'h0000_0000);
    
    @(posedge clock_signal);
    @(posedge clock_signal);

    $display();
    $display("=== Custom Data Types Test Complete ===");
    $display("Demonstrated usage of:");
    $display("- byte_t  (8-bit)  type alias");
    $display("- word_t  (16-bit) type alias");
    $display("- dword_t (32-bit) type alias");
    $display("Benefits: Improved code readability and maintainability");
    $display();

    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Custom Data Types Example ===
Testing byte_t, word_t, and dword_t definitions

Reset released - starting data processing tests

Byte processing:  00 -> 00
Word processing:  0000 -> 0000
DWord processing: 00000000 -> 00000000
QWord combined:   0000000000000000
----------------------------------------
Byte processing:  0f -> 01
Word processing:  1234 -> 0000
DWord processing: abcdef01 -> ffffffff
QWord combined:   0000000000000000
----------------------------------------
Byte processing:  0f -> 10
Word processing:  1234 -> 2468
DWord processing: abcdef01 -> 543210fe
QWord combined:   abcdef0112340f00
----------------------------------------
Byte processing:  0f -> 10
Word processing:  1234 -> 2468
DWord processing: abcdef01 -> 543210fe
QWord combined:   abcdef0112340f00
----------------------------------------
Byte processing:  ff -> 10
Word processing:  ffff -> 2468
DWord processing: ffffffff -> 543210fe
QWord combined:   abcdef0112340f00
---------------

0

### **Example 14: Array Type Shortcuts**
Defining reusable array types to avoid repeating complex array declarations.

In [14]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_14__array_type_shortcuts/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// array_type_shortcuts_design.sv - Array Type Shortcuts Example
module memory_controller_design ();

  // Define reusable array type shortcuts to avoid repetition
  typedef logic [7:0] byte_data_type;              // 8-bit data type
  typedef byte_data_type memory_word_array_type [0:15]; // 16-byte array
  typedef logic [31:0] address_bus_type [0:7];     // 8-word address array
  typedef struct packed {
    logic valid;
    logic [2:0] priority_level;
    logic [3:0] channel_id;
  } control_packet_type;
  typedef control_packet_type packet_buffer_type [0:3]; // 4-packet buffer

  // Declare variables using the type shortcuts
  memory_word_array_type cache_line_buffer;        // Cache line storage
  memory_word_array_type write_data_buffer;        // Write data buffer
  address_bus_type memory_address_queue;           // Address queue
  address_bus_type pending_address_list;           // Pending addresses
  packet_buffer_type input_packet_fifo;            // Input packet FIFO
  packet_buffer_type output_packet_fifo;           // Output packet FIFO

  initial begin
    $display("Array Type Shortcuts Demonstration");
    $display("=====================================");
    
    // Initialize arrays using the type shortcuts
    for (int i = 0; i < 16; i++) begin
      cache_line_buffer[i] = 8'(i * 16);            // Fill cache buffer
      write_data_buffer[i] = 8'(i + 128);           // Fill write buffer
    end
    
    for (int j = 0; j < 8; j++) begin
      memory_address_queue[j] = 32'h1000 + (j * 4); // Address sequence
      pending_address_list[j] = 32'h2000 + (j * 8); // Pending list
    end
    
    for (int k = 0; k < 4; k++) begin
      input_packet_fifo[k].valid = 1;           // Mark packets valid
      input_packet_fifo[k].priority_level = 3'(k); // Set priority level
      input_packet_fifo[k].channel_id = 4'(k + 1); // Set channel ID
      
      output_packet_fifo[k].valid = 0;          // Mark output empty
      output_packet_fifo[k].priority_level = 3'b0; // Clear priority level
      output_packet_fifo[k].channel_id = 4'b0;     // Clear channel
    end
    
    $display("Cache buffer[0:3]: %h %h %h %h", 
             cache_line_buffer[0], cache_line_buffer[1],
             cache_line_buffer[2], cache_line_buffer[3]);
    $display("Address queue[0:3]: %h %h %h %h",
             memory_address_queue[0], memory_address_queue[1],
             memory_address_queue[2], memory_address_queue[3]);
    $display("Packet FIFO[0] - Valid:%b Priority:%d Channel:%d",
             input_packet_fifo[0].valid, input_packet_fifo[0].priority_level,
             input_packet_fifo[0].channel_id);
  end

endmodule
```

```systemverilog
// array_type_shortcuts_testbench.sv
module memory_controller_testbench;

  // Reuse the same type shortcuts in testbench for consistency
  typedef logic [7:0] byte_data_type;
  typedef byte_data_type memory_word_array_type [0:15];
  typedef logic [31:0] address_bus_type [0:7];
  typedef struct packed {
    logic valid;
    logic [2:0] priority_level;
    logic [3:0] channel_id;
  } control_packet_type;
  typedef control_packet_type packet_buffer_type [0:3];

  // Testbench arrays using type shortcuts
  memory_word_array_type expected_cache_data;      // Expected results
  memory_word_array_type actual_cache_data;        // Actual results
  address_bus_type reference_addresses;            // Reference addresses
  packet_buffer_type test_packet_sequence;         // Test packets

  // Instantiate design under test
  memory_controller_design MEMORY_CTRL_INSTANCE();

  initial begin
    // Configure wave dumping
    $dumpfile("memory_controller_testbench.vcd");
    $dumpvars(0, memory_controller_testbench);
    
    #1; // Wait for design initialization
    
    $display();
    $display("Testbench: Array Type Shortcuts Verification");
    $display("============================================");
    
    // Initialize testbench arrays using type shortcuts
    for (int i = 0; i < 16; i++) begin
      expected_cache_data[i] = 8'(i * 16);      // Expected cache values
      actual_cache_data[i] = 8'(i * 16);        // Simulate actual values
    end
    
    for (int j = 0; j < 8; j++) begin
      reference_addresses[j] = 32'h1000 + (j * 4); // Reference sequence
    end
    
    for (int k = 0; k < 4; k++) begin
      test_packet_sequence[k].valid = (k % 2 == 0); // Alternate validity
      test_packet_sequence[k].priority_level = 3'(7 - k); // Descending priority
      test_packet_sequence[k].channel_id = 4'(k * 2);     // Even channel IDs
    end
    
    // Verify using type shortcuts makes code cleaner
    $display("Expected cache[4:7]: %h %h %h %h",
             expected_cache_data[4], expected_cache_data[5],
             expected_cache_data[6], expected_cache_data[7]);
    $display("Reference addr[4:7]: %h %h %h %h",
             reference_addresses[4], reference_addresses[5],
             reference_addresses[6], reference_addresses[7]);
    
    // Test packet verification
    for (int p = 0; p < 4; p++) begin
      $display("Test packet[%0d] - Valid:%b Priority:%d Channel:%d", p,
               test_packet_sequence[p].valid, test_packet_sequence[p].priority_level,
               test_packet_sequence[p].channel_id);
    end
    
    $display();
    $display("Benefits of array type shortcuts:");
    $display("- Consistent type definitions across modules");
    $display("- Reduced code duplication and errors");
    $display("- Easier maintenance and modifications");
    $display("- Improved code readability");
    $display();
    
  end

endmodule
```

Verilator Simulation Output:
Array Type Shortcuts Demonstration
Cache buffer[0:3]: 00 10 20 30
Address queue[0:3]: 00001000 00001004 00001008 0000100c
Packet FIFO[0] - Valid:1 Priority:0 Channel: 1

Testbench: Array Type Shortcuts Verification
Expected cache[4:7]: 40 50 60 70
Reference addr[4:7]: 00001010 00001014 00001018 0000101c
Test packet[0] - Valid:1 Priority:7 Channel: 0
Test packet[1] - Valid:0 Priority:6 Channel: 2
Test packet[2] - Valid:1 Priority:5 Channel: 4
Test packet[3] - Valid:0 Priority:4 Channel: 6

Benefits of array type shortcuts:
- Consistent type definitions across modules
- Reduced code duplication and errors
- Easier maintenance and modifications
- Improved code readability
Process finished with return code: 0
Removing Chapter_8_examples/example_14__array_type_shortcuts/obj_dir directory...
Chapter_8_examples/example_14__array_type_shortcuts/obj_dir removed successfully.


0

## Enumerated Types

Enumerated types provide a way to define a set of named constants, improving code readability and reducing errors.

### Basic Enumerations

```systemverilog
// Basic enumeration
typedef enum {RED, GREEN, BLUE} color_e;

// Enumeration with explicit values
typedef enum int {
    IDLE   = 0,
    ACTIVE = 1,
    WAIT   = 2,
    ERROR  = 9
} state_e;

// Enumeration with specific bit width
typedef enum logic [1:0] {
    READ  = 2'b00,
    WRITE = 2'b01,
    EXEC  = 2'b10,
    DEBUG = 2'b11
} operation_e;

module enum_examples;
    color_e pixel_color;
    state_e current_state, next_state;
    operation_e cpu_op;
    
    initial begin
        // Basic enumeration usage
        pixel_color = RED;
        $display("Pixel color: %s (value: %0d)", pixel_color.name(), pixel_color);
        
        // State machine example
        current_state = IDLE;
        
        case (current_state)
            IDLE: begin
                $display("Currently idle");
                next_state = ACTIVE;
            end
            ACTIVE: begin
                $display("Currently active");
                next_state = WAIT;
            end
            WAIT: begin
                $display("Currently waiting");
                next_state = IDLE;
            end
            ERROR: begin
                $display("Error state");
                next_state = IDLE;
            end
        endcase
        
        $display("Next state: %s", next_state.name());
        
        // Operation enumeration
        cpu_op = WRITE;
        $display("CPU operation: %s (encoding: %b)", cpu_op.name(), cpu_op);
        
        // Enumeration methods
        $display("First color: %s", color_e.first());
        $display("Last color: %s", color_e.last());
        $display("Number of colors: %0d", color_e.num());
    end
endmodule
```

### Advanced Enumeration Features

```systemverilog
// Enumeration with type specification
typedef enum bit [3:0] {
    CMD_NOP    = 4'h0,
    CMD_READ   = 4'h1,
    CMD_WRITE  = 4'h2,
    CMD_BURST  = 4'h4,
    CMD_RESET  = 4'hF
} command_e;

// Enumeration for one-hot encoding
typedef enum logic [7:0] {
    STAGE_0 = 8'b00000001,
    STAGE_1 = 8'b00000010,
    STAGE_2 = 8'b00000100,
    STAGE_3 = 8'b00001000,
    STAGE_4 = 8'b00010000,
    STAGE_5 = 8'b00100000,
    STAGE_6 = 8'b01000000,
    STAGE_7 = 8'b10000000
} pipeline_stage_e;

module advanced_enums;
    command_e cmd;
    pipeline_stage_e stage;
    
    initial begin
        // Command enumeration
        cmd = CMD_READ;
        $display("Command: %s (0x%h)", cmd.name(), cmd);
        
        // Check valid enumeration values
        cmd = command_e'(4'h3); // Invalid value
        if (cmd.name() == "") begin
            $display("Invalid command value: 0x%h", cmd);
        end
        
        // Pipeline stage (one-hot)
        stage = STAGE_3;
        $display("Pipeline stage: %s (0b%08b)", stage.name(), stage);
        
        // Iterate through enumeration values
        command_e cmd_iter = cmd.first();
        $display("\nAll valid commands:");
        do begin
            $display("  %s = 0x%h", cmd_iter.name(), cmd_iter);
            cmd_iter = cmd_iter.next();
        end while (cmd_iter != cmd.first());
        
        // Enumeration in arrays
        command_e cmd_history[$];
        cmd_history = {CMD_NOP, CMD_READ, CMD_WRITE, CMD_BURST};
        
        $display("\nCommand history:");
        foreach(cmd_history[i]) begin
            $display("  [%0d]: %s", i, cmd_history[i].name());
        end
    end
endmodule
```

### **Example 15: Traffic Light States**
Basic enum with RED, YELLOW, GREEN states for a simple state machine.

In [15]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_15__traffic_light_states/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// traffic_light_controller.sv
module traffic_light_controller (
    input logic clk,
    input logic reset_n,
    output logic [2:0] light_outputs  // RED, YELLOW, GREEN
);

  // Define traffic light states using enum
  typedef enum logic [1:0] {
    RED    = 2'b00,
    YELLOW = 2'b01,
    GREEN  = 2'b10
  } traffic_state_t;

  traffic_state_t current_state, next_state;
  logic [3:0] counter;  // Timer counter for state transitions

  // State register
  always_ff @(posedge clk or negedge reset_n) begin
    if (!reset_n) begin
      current_state <= RED;
      counter <= 4'b0000;
    end else begin
      current_state <= next_state;
      if (counter == 4'b1111)
        counter <= 4'b0000;
      else
        counter <= counter + 1;
    end
  end

  // Next state logic
  always_comb begin
    next_state = current_state;  // Default: stay in current state
    
    case (current_state)
      RED: begin
        if (counter == 4'b1111)  // Stay in RED for 16 cycles
          next_state = GREEN;
      end
      
      YELLOW: begin
        if (counter == 4'b0111)  // Stay in YELLOW for 8 cycles
          next_state = RED;
      end
      
      GREEN: begin
        if (counter == 4'b1111)  // Stay in GREEN for 16 cycles
          next_state = YELLOW;
      end
      
      default: next_state = RED;
    endcase
  end

  // Output logic
  always_comb begin
    light_outputs = 3'b000;  // All lights off by default
    
    case (current_state)
      RED:    light_outputs = 3'b100;  // Only RED on
      YELLOW: light_outputs = 3'b010;  // Only YELLOW on
      GREEN:  light_outputs = 3'b001;  // Only GREEN on
      default: light_outputs = 3'b100; // Default to RED
    endcase
  end

  // Display current state for debugging
  initial begin
    $display("Traffic Light Controller initialized");
    $display("State encoding: RED=00, YELLOW=01, GREEN=10");
  end

endmodule
```

```systemverilog
// traffic_light_controller_testbench.sv
module traffic_light_testbench;

  // Testbench signals
  logic clk;
  logic reset_n;
  logic [2:0] light_outputs;

  // Instantiate design under test
  traffic_light_controller TRAFFIC_LIGHT_DUT (
    .clk(clk),
    .reset_n(reset_n),
    .light_outputs(light_outputs)
  );

  // Clock generation
  initial begin
    clk = 0;
    forever #5 clk = ~clk;  // 10ns period clock
  end

  // Test stimulus
  initial begin
    // Dump waves
    $dumpfile("traffic_light_testbench.vcd");
    $dumpvars(0, traffic_light_testbench);

    // Initialize signals
    reset_n = 0;
    
    $display();
    $display("=== Traffic Light Controller Testbench ===");
    $display("Time\tReset\tState\tRED\tYELLOW\tGREEN");
    $display("----\t-----\t-----\t---\t------\t-----");

    // Apply reset
    #20 reset_n = 1;
    
    // Monitor traffic light states
    repeat (200) begin
      @(posedge clk);
      $display("%4t\t%b\t%s\t%b\t%b\t%b", 
               $time, reset_n, 
               get_state_name(TRAFFIC_LIGHT_DUT.current_state),
               light_outputs[2], light_outputs[1], light_outputs[0]);
    end

    $display();
    $display("Traffic light simulation completed successfully!");
    $display();
    $finish;
  end

  // Function to convert state enum to string for display
  function string get_state_name(logic [1:0] state);
    case (state)
      2'b00: return "RED   ";
      2'b01: return "YELLOW";
      2'b10: return "GREEN ";
      default: return "UNKNWN";
    endcase
  endfunction

  // Monitor state changes
  always @(posedge clk) begin
    if (reset_n && TRAFFIC_LIGHT_DUT.current_state != 
        TRAFFIC_LIGHT_DUT.next_state) begin
      $display("State transition: %s -> %s at time %t",
               get_state_name(TRAFFIC_LIGHT_DUT.current_state),
               get_state_name(TRAFFIC_LIGHT_DUT.next_state),
               $time);
    end
  end

endmodule
```

Verilator Simulation Output:
Traffic Light Controller initialized
State encoding: RED=00, YELLOW=01, GREEN=10

=== Traffic Light Controller Testbench ===
Time	Reset	State	RED	YELLOW	GREEN
----	-----	-----	---	------	-----
  25	1	RED   	1	0	0
  35	1	RED   	1	0	0
  45	1	RED   	1	0	0
  55	1	RED   	1	0	0
  65	1	RED   	1	0	0
  75	1	RED   	1	0	0
  85	1	RED   	1	0	0
  95	1	RED   	1	0	0
 105	1	RED   	1	0	0
 115	1	RED   	1	0	0
 125	1	RED   	1	0	0
 135	1	RED   	1	0	0
 145	1	RED   	1	0	0
 155	1	RED   	1	0	0
 165	1	RED   	1	0	0
 175	1	RED   	1	0	0
State transition: RED    -> GREEN  at time                  175
 185	1	GREEN 	0	0	1
 195	1	GREEN 	0	0	1
 205	1	GREEN 	0	0	1
 215	1	GREEN 	0	0	1
 225	1	GREEN 	0	0	1
 235	1	GREEN 	0	0	1
 245	1	GREEN 	0	0	1
 255	1	GREEN 	0	0	1
 265	1	GREEN 	0	0	1
 275	1	GREEN 	0	0	1
 285	1	GREEN 	0	0	1
 295	1	GREEN 	0	0	1
 305	1	GREEN 	0	0	1
 315	1	GREEN 	0	0	1
 325	1	GREEN 	0	0	1
 335	1	GREEN 	0	0	1
State transition: GREEN  -> YELLOW at time                  335
 345	1	YEL

0

### **Example 16: Memory Access Types**
Enum defining READ, WRITE, EXECUTE operations with specific bit encodings.

In [16]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_16__memory_access_types/"
read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// memory_access_controller.sv
module memory_access_controller #(
  parameter DATA_WIDTH = 32,
  parameter ADDR_WIDTH = 16
)(
  input  logic                    clk,
  input  logic                    reset_n,
  input  logic [ADDR_WIDTH-1:0]   address,
  input  logic [DATA_WIDTH-1:0]   write_data,
  input  logic [2:0]              access_type,
  input  logic                    access_enable,
  output logic [DATA_WIDTH-1:0]   read_data,
  output logic                    access_valid,
  output logic                    access_error
);

  // Memory access type constants with specific bit encodings
  localparam logic [2:0] MEM_READ    = 3'b001;  // Read operation    - bit 0 set
  localparam logic [2:0] MEM_WRITE   = 3'b010;  // Write operation   - bit 1 set
  localparam logic [2:0] MEM_EXECUTE = 3'b100;  // Execute operation - bit 2 set

  // Internal memory array for simulation
  logic [DATA_WIDTH-1:0] memory_array [0:2**ADDR_WIDTH-1];
  
  // Access control logic
  always_ff @(posedge clk or negedge reset_n) begin
    if (!reset_n) begin
      read_data    <= '0;
      access_valid <= 1'b0;
      access_error <= 1'b0;
    end else begin
      access_valid <= 1'b0;
      access_error <= 1'b0;
      
      if (access_enable) begin
        case (access_type)
          MEM_READ: begin
            read_data    <= memory_array[address];
            access_valid <= 1'b1;
            $display("Time: %0t - READ from addr 0x%04h, data: 0x%08h", 
                     $time, address, memory_array[address]);
          end
          
          MEM_WRITE: begin
            memory_array[address] <= write_data;
            access_valid         <= 1'b1;
            $display("Time: %0t - WRITE to addr 0x%04h, data: 0x%08h", 
                     $time, address, write_data);
          end
          
          MEM_EXECUTE: begin
            read_data    <= memory_array[address];
            access_valid <= 1'b1;
            $display("Time: %0t - EXECUTE from addr 0x%04h, instr: 0x%08h", 
                     $time, address, memory_array[address]);
          end
          
          default: begin
            access_error <= 1'b1;
            $display("Time: %0t - ERROR: Invalid access type: %b", 
                     $time, access_type);
          end
        endcase
      end
    end
  end

  // Initialize memory with some test data
  initial begin
    memory_array[16'h0000] = 32'hDEADBEEF;  // Test data
    memory_array[16'h0001] = 32'hCAFEBABE;  // Test data
    memory_array[16'h0002] = 32'h12345678;  // Test instruction
    memory_array[16'h0003] = 32'h9ABCDEF0;  // Test instruction
  end

endmodule
```

```systemverilog
// memory_access_controller_testbench.sv
module memory_access_testbench;

  // Memory access type constants (must match design)
  localparam logic [2:0] MEM_READ    = 3'b001;  // Read operation    - bit 0 set
  localparam logic [2:0] MEM_WRITE   = 3'b010;  // Write operation   - bit 1 set
  localparam logic [2:0] MEM_EXECUTE = 3'b100;  // Execute operation - bit 2 set

  // Testbench signals
  logic                    clk;
  logic                    reset_n;
  logic [15:0]             address;
  logic [31:0]             write_data;
  logic [2:0]              access_type;
  logic                    access_enable;
  logic [31:0]             read_data;
  logic                    access_valid;
  logic                    access_error;

  // Instantiate the design under test
  memory_access_controller #(
    .DATA_WIDTH(32),
    .ADDR_WIDTH(16)
  ) MEMORY_CONTROLLER_INST (
    .clk(clk),
    .reset_n(reset_n),
    .address(address),
    .write_data(write_data),
    .access_type(access_type),
    .access_enable(access_enable),
    .read_data(read_data),
    .access_valid(access_valid),
    .access_error(access_error)
  );

  // Clock generation
  initial begin
    clk = 1'b0;
    forever #5 clk = ~clk;  // 10ns period (100MHz)
  end

  // Test sequence
  initial begin
    // Initialize waveform dump
    $dumpfile("memory_access_testbench.vcd");
    $dumpvars(0, memory_access_testbench);
    
    $display();
    $display("=== Memory Access Types Test ===");
    $display();
    
    // Initialize signals
    reset_n      = 1'b0;
    address      = 16'h0000;
    write_data   = 32'h00000000;
    access_type  = MEM_READ;
    access_enable = 1'b0;
    
    // Reset sequence
    #20 reset_n = 1'b1;
    #10;
    
    // Test READ operation
    $display("--- Testing READ Operation ---");
    test_memory_access(16'h0000, 32'h00000000, MEM_READ);
    test_memory_access(16'h0001, 32'h00000000, MEM_READ);
    
    // Test WRITE operation
    $display("--- Testing WRITE Operation ---");
    test_memory_access(16'h0100, 32'hABCD1234, MEM_WRITE);
    test_memory_access(16'h0101, 32'h5678CDEF, MEM_WRITE);
    
    // Test READ after WRITE
    $display("--- Testing READ after WRITE ---");
    test_memory_access(16'h0100, 32'h00000000, MEM_READ);
    test_memory_access(16'h0101, 32'h00000000, MEM_READ);
    
    // Test EXECUTE operation
    $display("--- Testing EXECUTE Operation ---");
    test_memory_access(16'h0002, 32'h00000000, MEM_EXECUTE);
    test_memory_access(16'h0003, 32'h00000000, MEM_EXECUTE);
    
    // Test invalid access type
    $display("--- Testing Invalid Access Type ---");
    address      = 16'h0000;
    access_type  = 3'b111;  // Invalid type
    access_enable = 1'b1;
    @(posedge clk);
    access_enable = 1'b0;
    @(posedge clk);
    
    $display();
    $display("=== Test Complete ===");
    $display();
    
    #50 $finish;
  end

  // Task to perform memory access test
  task test_memory_access(
    input [15:0] addr,
    input [31:0] wdata,
    input [2:0] acc_type
  );
    begin
      address      = addr;
      write_data   = wdata;
      access_type  = acc_type;
      access_enable = 1'b1;
      
      @(posedge clk);
      access_enable = 1'b0;
      
      @(posedge clk);
      
      // Check results
      if (access_valid && !access_error) begin
        $display("âœ“ Access successful");
      end else if (access_error) begin
        $display("âœ— Access error detected");
      end else begin
        $display("? Access pending");
      end
      $display();
    end
  endtask

endmodule
```

Verilator Simulation Output:

=== Memory Access Types Test ===

--- Testing READ Operation ---
? Access pending

Time: 45 - READ from addr 0x0001, data: 0xcafebabe
? Access pending

--- Testing WRITE Operation ---
Time: 65 - WRITE to addr 0x0100, data: 0xabcd1234
? Access pending

Time: 85 - WRITE to addr 0x0101, data: 0x5678cdef
? Access pending

--- Testing READ after WRITE ---
Time: 105 - READ from addr 0x0100, data: 0xabcd1234
? Access pending

Time: 125 - READ from addr 0x0101, data: 0x5678cdef
? Access pending

--- Testing EXECUTE Operation ---
Time: 145 - EXECUTE from addr 0x0002, instr: 0x12345678
? Access pending

Time: 165 - EXECUTE from addr 0x0003, instr: 0x9abcdef0
? Access pending

--- Testing Invalid Access Type ---
Time: 185 - ERROR: Invalid access type: 111

=== Test Complete ===

Process finished with return code: 0
Removing Chapter_8_examples/example_16__memory_access_types/obj_dir directory...
Chapter_8_examples/example_16__memory_access_types/obj_dir removed succes

0

### **Example 17: Error Code Definitions**
Enumeration of common error codes with meaningful names instead of magic numbers.

In [17]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_17__error_code_definitions/"
read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// error_code_processor.sv
module error_code_processor (
  input  logic        clk,
  input  logic        reset_n,
  input  logic [2:0]  operation,
  input  logic [7:0]  data_a,
  input  logic [7:0]  data_b,
  output logic [7:0]  result,
  output logic [3:0]  error_code
);

  // Error code enumeration with meaningful names
  typedef enum logic [3:0] {
    ERR_NONE         = 4'h0,  // No error
    ERR_DIVIDE_ZERO  = 4'h1,  // Division by zero
    ERR_OVERFLOW     = 4'h2,  // Arithmetic overflow
    ERR_UNDERFLOW    = 4'h3,  // Arithmetic underflow
    ERR_INVALID_OP   = 4'h4,  // Invalid operation
    ERR_OUT_OF_RANGE = 4'h5,  // Input out of range
    ERR_TIMEOUT      = 4'h6,  // Operation timeout
    ERR_UNKNOWN      = 4'hF   // Unknown error
  } error_code_t;

  // Operation codes
  typedef enum logic [2:0] {
    OP_ADD     = 3'b000,
    OP_SUB     = 3'b001,
    OP_MUL     = 3'b010,
    OP_DIV     = 3'b011,
    OP_MOD     = 3'b100,
    OP_SQRT    = 3'b101
  } operation_t;

  error_code_t current_error;
  logic [15:0] temp_result;

  always_ff @(posedge clk or negedge reset_n) begin
    if (!reset_n) begin
      result        <= 8'h00;
      current_error <= ERR_NONE;
    end else begin
      current_error <= ERR_NONE;  // Default to no error
      
      case (operation)
        OP_ADD: begin
          temp_result = {8'h00, data_a} + {8'h00, data_b};
          if (temp_result > 255) begin
            current_error <= ERR_OVERFLOW;
            result <= 8'hFF;  // Saturate to max value
          end else begin
            result <= temp_result[7:0];
          end
        end
        
        OP_SUB: begin
          if (data_a < data_b) begin
            current_error <= ERR_UNDERFLOW;
            result <= 8'h00;  // Saturate to min value
          end else begin
            result <= data_a - data_b;
          end
        end
        
        OP_MUL: begin
          temp_result = {8'h00, data_a} * {8'h00, data_b};
          if (temp_result > 255) begin
            current_error <= ERR_OVERFLOW;
            result <= 8'hFF;  // Saturate to max value
          end else begin
            result <= temp_result[7:0];
          end
        end
        
        OP_DIV: begin
          if (data_b == 0) begin
            current_error <= ERR_DIVIDE_ZERO;
            result <= 8'h00;  // Return 0 for divide by zero
          end else begin
            result <= data_a / data_b;
          end
        end
        
        OP_MOD: begin
          if (data_b == 0) begin
            current_error <= ERR_DIVIDE_ZERO;
            result <= 8'h00;  // Return 0 for divide by zero
          end else begin
            result <= data_a % data_b;
          end
        end
        
        OP_SQRT: begin
          if (data_a > 225) begin // sqrt(225) = 15, max for 8-bit result
            current_error <= ERR_OUT_OF_RANGE;
            result <= 8'h00;  // Return 0 for out of range
          end else begin
            result <= 8'h00;  // Simplified, just return 0
          end
        end
        
        default: begin
          current_error <= ERR_INVALID_OP;
          result <= 8'h00;
        end
      endcase
    end
  end

  assign error_code = current_error;

endmodule
```

```systemverilog
// error_code_processor_testbench.sv
module error_code_processor_testbench;

  // Clock and reset
  logic        clk;
  logic        reset_n;
  
  // DUT signals
  logic [2:0]  operation;
  logic [7:0]  data_a;
  logic [7:0]  data_b;
  logic [7:0]  result;
  logic [3:0]  error_code;

  // Error code names for display
  string error_name;
  
  // Instantiate design under test
  error_code_processor DUT (
    .clk(clk),
    .reset_n(reset_n),
    .operation(operation),
    .data_a(data_a),
    .data_b(data_b),
    .result(result),
    .error_code(error_code)
  );

  // Clock generation
  initial begin
    clk = 0;
    forever #5 clk = ~clk;
  end

  // Function to convert error code to readable name
  function string get_error_name(logic [3:0] code);
    case (code)
      4'h0: return "ERR_NONE";
      4'h1: return "ERR_DIVIDE_ZERO";
      4'h2: return "ERR_OVERFLOW";
      4'h3: return "ERR_UNDERFLOW";
      4'h4: return "ERR_INVALID_OP";
      4'h5: return "ERR_OUT_OF_RANGE";
      4'h6: return "ERR_TIMEOUT";
      4'hF: return "ERR_UNKNOWN";
      default: return "UNDEFINED";
    endcase
  endfunction

  // Test stimulus
  initial begin
    // Dump waves
    $dumpfile("error_code_processor_testbench.vcd");
    $dumpvars(0, error_code_processor_testbench);
    
    $display();
    $display("=== Error Code Definitions Test ===");
    $display();
    
    // Initialize
    reset_n = 0;
    operation = 3'b000;
    data_a = 8'h00;
    data_b = 8'h00;
    
    // Reset
    #10 reset_n = 1;
    #10;
    
    // Test 1: Normal addition (no error)
    operation = 3'b000;  // ADD
    data_a = 8'h10;
    data_b = 8'h05;
    #10;
    error_name = get_error_name(error_code);
    $display("Test 1 - ADD: %0d + %0d = %0d, Error: %s", 
             data_a, data_b, result, error_name);
    
    // Test 2: Addition overflow
    operation = 3'b000;  // ADD
    data_a = 8'hFF;
    data_b = 8'h01;
    #10;
    error_name = get_error_name(error_code);
    $display("Test 2 - ADD Overflow: %0d + %0d = %0d, Error: %s", 
             data_a, data_b, result, error_name);
    
    // Test 3: Subtraction underflow
    operation = 3'b001;  // SUB
    data_a = 8'h05;
    data_b = 8'h10;
    #10;
    error_name = get_error_name(error_code);
    $display("Test 3 - SUB Underflow: %0d - %0d = %0d, Error: %s", 
             data_a, data_b, result, error_name);
    
    // Test 4: Division by zero
    operation = 3'b011;  // DIV
    data_a = 8'h20;
    data_b = 8'h00;
    #10;
    error_name = get_error_name(error_code);
    $display("Test 4 - DIV by Zero: %0d / %0d = %0d, Error: %s", 
             data_a, data_b, result, error_name);
    
    // Test 5: Multiplication overflow
    operation = 3'b010;  // MUL
    data_a = 8'h20;
    data_b = 8'h10;
    #10;
    error_name = get_error_name(error_code);
    $display("Test 5 - MUL Overflow: %0d * %0d = %0d, Error: %s", 
             data_a, data_b, result, error_name);
    
    // Test 6: Invalid operation
    operation = 3'b111;  // Invalid
    data_a = 8'h10;
    data_b = 8'h05;
    #10;
    error_name = get_error_name(error_code);
    $display("Test 6 - Invalid Op: op=%b, Error: %s", 
             operation, error_name);
    
    // Test 7: Out of range (SQRT)
    operation = 3'b101;  // SQRT
    data_a = 8'hFF;      // Too large for sqrt
    data_b = 8'h00;
    #10;
    error_name = get_error_name(error_code);
    $display("Test 7 - Out of Range: sqrt(%0d), Error: %s", 
             data_a, error_name);
    
    $display();
    $display("=== Error Code Test Complete ===");
    $display();
    
    #20 $finish;
  end

endmodule
```

Verilator Simulation Output:

=== Error Code Definitions Test ===

Test 1 - ADD: 16 + 5 = 21, Error: ERR_NONE
Test 2 - ADD Overflow: 255 + 1 = 255, Error: ERR_OVERFLOW
Test 3 - SUB Underflow: 5 - 16 = 0, Error: ERR_UNDERFLOW
Test 4 - DIV by Zero: 32 / 0 = 0, Error: ERR_DIVIDE_ZERO
Test 5 - MUL Overflow: 32 * 16 = 255, Error: ERR_OVERFLOW
Test 6 - Invalid Op: op=111, Error: ERR_INVALID_OP
Test 7 - Out of Range: sqrt(255), Error: ERR_OUT_OF_RANGE

=== Error Code Test Complete ===

Process finished with return code: 0
Removing Chapter_8_examples/example_17__error_code_definitions/obj_dir directory...
Chapter_8_examples/example_17__error_code_definitions/obj_dir removed successfully.


0

## User-Defined Types

SystemVerilog allows you to create sophisticated user-defined types combining structures, unions, and other advanced features.

### Complex Structure Types

```systemverilog
// Memory transaction structure
typedef struct {
    logic [31:0] address;
    logic [31:0] data;
    logic [3:0]  byte_enable;
    logic        read_write;  // 0=read, 1=write
    logic        valid;
    int          timestamp;
} memory_transaction_t;

// Network packet structure
typedef struct {
    logic [47:0] dest_mac;
    logic [47:0] src_mac;
    logic [15:0] ether_type;
    logic [7:0]  payload[$];  // Variable size payload
    logic [31:0] crc;
} ethernet_packet_t;

// Configuration register structure
typedef struct packed {
    logic        enable;
    logic [2:0]  mode;
    logic        interrupt_enable;
    logic        auto_increment;
    logic [1:0]  priority;
    logic [24:0] reserved;
} config_register_t;

module user_defined_types;
    memory_transaction_t mem_trans;
    ethernet_packet_t eth_packet;
    config_register_t config_reg;
    
    initial begin
        // Memory transaction example
        mem_trans.address = 32'h1000_0000;
        mem_trans.data = 32'hDEAD_BEEF;
        mem_trans.byte_enable = 4'b1111;
        mem_trans.read_write = 1'b1;  // Write
        mem_trans.valid = 1'b1;
        mem_trans.timestamp = $time;
        
        $display("Memory Transaction:");
        $display("  Address: 0x%h", mem_trans.address);
        $display("  Data: 0x%h", mem_trans.data);
        $display("  Operation: %s", mem_trans.read_write ? "WRITE" : "READ");
        $display("  Timestamp: %0t", mem_trans.timestamp);
        
        // Ethernet packet example
        eth_packet.dest_mac = 48'hFF_FF_FF_FF_FF_FF;  // Broadcast
        eth_packet.src_mac = 48'h00_11_22_33_44_55;
        eth_packet.ether_type = 16'h0800;  // IPv4
        
        // Add payload data
        for (int i = 0; i < 64; i++) begin
            eth_packet.payload.push_back(8'(i));
        end
        
        eth_packet.crc = 32'h12345678;
        
        $display("\nEthernet Packet:");
        $display("  Destination MAC: %012h", eth_packet.dest_mac);
        $display("  Source MAC: %012h", eth_packet.src_mac);
        $display("  EtherType: 0x%04h", eth_packet.ether_type);
        $display("  Payload size: %0d bytes", eth_packet.payload.size());
        
        // Configuration register example
        config_reg = '{
            enable: 1'b1,
            mode: 3'b101,
            interrupt_enable: 1'b1,
            auto_increment: 1'b0,
            priority: 2'b10,
            reserved: 25'b0
        };
        
        $display("\nConfiguration Register (0x%08h):", config_reg);
        $display("  Enable: %b", config_reg.enable);
        $display("  Mode: %b", config_reg.mode);
        $display("  Interrupt Enable: %b", config_reg.interrupt_enable);
        $display("  Priority: %b", config_reg.priority);
    end
endmodule
```

### **Example 18: Simple Transaction**
Structure combining address, data, and control signals for basic bus transactions.

In [18]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_18__simple_transaction/"
read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// simple_bus_transaction.sv
// Simple transaction structure for basic bus operations

// Transaction class containing address, data, and control signals
class simple_bus_transaction;
  rand bit [31:0] address;        // 32-bit address field
  rand bit [31:0] data;           // 32-bit data field  
  rand bit        write_enable;   // Control signal: 1=write, 0=read
  rand bit        valid;          // Transaction valid signal
  
  // Constructor
  function new();
    address = 0;
    data = 0;
    write_enable = 0;
    valid = 0;
  endfunction
  
  // Display transaction contents
  function void display(string prefix = "");
    $display("%s Transaction:", prefix);
    $display("  Address: 0x%08h", address);
    $display("  Data:    0x%08h", data);
    $display("  Write:   %b", write_enable);
    $display("  Valid:   %b", valid);
    $display();
  endfunction
  
  // Copy transaction
  function simple_bus_transaction copy();
    simple_bus_transaction copied_trans = new();
    copied_trans.address = this.address;
    copied_trans.data = this.data;
    copied_trans.write_enable = this.write_enable;
    copied_trans.valid = this.valid;
    return copied_trans;
  endfunction
  
  // Compare two transactions
  function bit compare(simple_bus_transaction other);
    return (this.address == other.address && 
            this.data == other.data &&
            this.write_enable == other.write_enable &&
            this.valid == other.valid);
  endfunction
  
endclass

// Simple bus interface for demonstration
interface simple_bus_interface;
  logic [31:0] address;
  logic [31:0] data;
  logic        write_enable;
  logic        valid;
  logic        ready;
  
  // Modport for master (drives transactions)
  modport master (
    output address, data, write_enable, valid,
    input  ready
  );
  
  // Modport for slave (receives transactions)
  modport slave (
    input  address, data, write_enable, valid,
    output ready
  );
  
endinterface

// Simple bus master module
module simple_bus_master (
  input  logic clk,
  input  logic reset_n,
  simple_bus_interface.master bus_if
);
  
  // Simple state machine for demonstration
  typedef enum logic [1:0] {
    IDLE,
    SEND_TRANS,
    WAIT_READY
  } state_t;
  
  state_t current_state, next_state;
  
  // State register
  always_ff @(posedge clk or negedge reset_n) begin
    if (!reset_n)
      current_state <= IDLE;
    else
      current_state <= next_state;
  end
  
  // Next state logic (simplified for example)
  always_comb begin
    next_state = current_state;
    case (current_state)
      IDLE: next_state = SEND_TRANS;
      SEND_TRANS: next_state = WAIT_READY;
      WAIT_READY: if (bus_if.ready) next_state = IDLE;
      default: next_state = IDLE;  // Handle unassigned states
    endcase
  end
  
  // Output logic
  always_comb begin
    bus_if.address = 32'h0;
    bus_if.data = 32'h0;
    bus_if.write_enable = 1'b0;
    bus_if.valid = 1'b0;
    
    case (current_state)
      SEND_TRANS, WAIT_READY: begin
        bus_if.address = 32'hDEADBEEF;
        bus_if.data = 32'hCAFEBABE;
        bus_if.write_enable = 1'b1;
        bus_if.valid = 1'b1;
      end
      default: begin
        // Default case: all signals remain at reset values
        bus_if.address = 32'h0;
        bus_if.data = 32'h0;
        bus_if.write_enable = 1'b0;
        bus_if.valid = 1'b0;
      end
    endcase
  end
  
endmodule
```

```systemverilog
// simple_bus_transaction_testbench.sv
// Testbench for simple bus transaction example

module simple_bus_transaction_testbench;
  
  // Clock and reset generation
  logic clk;
  logic reset_n;
  
  // Clock generation
  initial begin
    clk = 0;
    forever #5 clk = ~clk;  // 10ns period clock
  end
  
  // Reset generation
  initial begin
    reset_n = 0;
    #25 reset_n = 1;
  end
  
  // Bus interface instance
  simple_bus_interface bus_if();
  
  // Simple slave ready signal generation
  initial begin
    bus_if.ready = 0;
    #50 bus_if.ready = 1;
    #20 bus_if.ready = 0;
    #30 bus_if.ready = 1;
  end
  
  // DUT instantiation
  simple_bus_master dut (
    .clk(clk),
    .reset_n(reset_n),
    .bus_if(bus_if.master)
  );
  
  // Transaction objects for testing
  simple_bus_transaction original_trans;
  simple_bus_transaction copied_trans;
  simple_bus_transaction compare_trans;
  
  // Main test sequence
  initial begin
    // Dump waves
    $dumpfile("simple_bus_transaction_testbench.vcd");
    $dumpvars(0, simple_bus_transaction_testbench);
    
    $display("=== Simple Bus Transaction Example ===");
    $display();
    
    // Wait for reset deassertion
    wait(reset_n);
    #10;
    
    // Create and configure original transaction
    original_trans = new();
    original_trans.address = 32'h1000_BEEF;
    original_trans.data = 32'hDEAD_CAFE;
    original_trans.write_enable = 1'b1;
    original_trans.valid = 1'b1;
    
    $display("Original transaction created:");
    original_trans.display("ORIG");
    
    // Test copy function
    copied_trans = original_trans.copy();
    $display("Copied transaction:");
    copied_trans.display("COPY");
    
    // Test compare function - should match
    if (original_trans.compare(copied_trans)) begin
      $display("Copy comparison: PASSED - Transactions match");
    end else begin
      $display("Copy comparison: FAILED - Transactions don't match");
    end
    $display();
    
    // Create different transaction for comparison test
    compare_trans = new();
    compare_trans.address = 32'h2000_BEEF;  // Different address
    compare_trans.data = 32'hDEAD_CAFE;
    compare_trans.write_enable = 1'b1;
    compare_trans.valid = 1'b1;
    
    $display("Different transaction for comparison:");
    compare_trans.display("DIFF");
    
    // Test compare function - should not match
    if (!original_trans.compare(compare_trans)) begin
      $display("Different comparison: PASSED - Transactions differ");
    end else begin
      $display("Different comparison: FAILED - Should not match");
    end
    $display();
    
    // Test randomization
    $display("Testing transaction randomization:");
    repeat (3) begin
      int rand_result;
      rand_result = original_trans.randomize();
      if (rand_result == 0) begin
        $display("Warning: Randomization failed");
      end
      original_trans.display("RAND");
    end
    
    // Monitor bus interface activity
    $display("Monitoring bus interface activity:");
    fork
      // Monitor bus signals
      begin
        forever begin
          @(posedge clk);
          if (bus_if.valid && bus_if.ready) begin
            $display("Bus Transaction Detected:");
            $display("  Time: %0t", $time);
            $display("  Addr: 0x%08h", bus_if.address);
            $display("  Data: 0x%08h", bus_if.data);
            $display("  Write: %b", bus_if.write_enable);
            $display();
          end
        end
      end
      
      // Test timeout
      begin
        #200;
      end
    join_any
    
    $display("=== Test Complete ===");
    $finish;
  end
  
  // Optional: Monitor all bus activity
  always @(posedge clk) begin
    if (bus_if.valid) begin
      $display("Time %0t: Bus Active - Addr:0x%h Data:0x%h WE:%b Ready:%b", 
               $time, bus_if.address, bus_if.data, 
               bus_if.write_enable, bus_if.ready);
    end
  end
  
endmodule
```

Verilator Simulation Output:
- Verilator: Walltime 31.206 s (elab=0.002, cvt=0.060, bld=30.132); cpu 0.054 s
on 1 threads; alloced 20.180 MB
=== Simple Bus Transaction Example ===

Original transaction created:
ORIG Transaction:
  Address: 0x1000beef
  Data:    0xdeadcafe
  Write:   1
  Valid:   1

Copied transaction:
COPY Transaction:
  Address: 0x1000beef
  Data:    0xdeadcafe
  Write:   1
  Valid:   1

Copy comparison: PASSED - Transactions match

Different transaction for comparison:
DIFF Transaction:
  Address: 0x2000beef
  Data:    0xdeadcafe
  Write:   1
  Valid:   1

Different comparison: PASSED - Transactions differ

Testing transaction randomization:
RAND Transaction:
  Address: 0x797673c4
  Data:    0x0b1e5d9c
  Write:   0
  Valid:   0

RAND Transaction:
  Address: 0x92178378
  Data:    0xbbd58ebc
  Write:   0
  Valid:   0

RAND Transaction:
  Address: 0x7d6ff3b7
  Data:    0xfcc47153
  Write:   1
  Valid:   1

Monitoring bus interface activity:
Time 35: Bus Active - Addr:0x

0

### **Example 19: Configuration Settings**
Packed structure representing hardware configuration registers with named fields.

In [19]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_19__config_registers/"
read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// hardware_config_registers.sv
module hardware_config_registers (
  input  logic        clk,
  input  logic        rst_n,
  input  logic        write_enable,
  input  logic [31:0] config_data_in,
  output logic [31:0] config_data_out,
  output logic        interrupt_enable,
  output logic [2:0]  priority_level,
  output logic [7:0]  device_id
);

  // Packed structure for configuration register
  typedef struct packed {
    logic [7:0]  reserved;        // Bits [31:24] - Reserved for future use
    logic [7:0]  device_id;       // Bits [23:16] - Device identification
    logic [4:0]  unused;          // Bits [15:11] - Unused bits
    logic [2:0]  priority_level;  // Bits [10:8]  - Interrupt priority
    logic [6:0]  feature_flags;   // Bits [7:1]   - Various feature enables
    logic        interrupt_enable; // Bit [0]     - Global interrupt enable
  } config_register_t;

  // Configuration register instance
  config_register_t config_reg;

  // Register update logic
  always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
      // Reset to default configuration
      config_reg.reserved        <= 8'h00;
      config_reg.device_id       <= 8'hA5;  // Default device ID
      config_reg.unused          <= 5'b00000;
      config_reg.priority_level  <= 3'b010; // Medium priority
      config_reg.feature_flags   <= 7'b0000001; // Basic features
      config_reg.interrupt_enable <= 1'b0;  // Interrupts disabled
    end else if (write_enable) begin
      // Write new configuration
      config_reg <= config_data_in;
    end
  end

  // Output assignments from packed structure fields
  assign config_data_out   = config_reg;
  assign interrupt_enable  = config_reg.interrupt_enable;
  assign priority_level    = config_reg.priority_level;
  assign device_id         = config_reg.device_id;

  // Display configuration changes
  always @(posedge clk) begin
    if (write_enable && rst_n) begin
      $display("Config Update: Device=0x%02X, Priority=%0d, IntEn=%b",
               config_reg.device_id, config_reg.priority_level,
               config_reg.interrupt_enable);
    end
  end

endmodule
```

```systemverilog
// hardware_config_registers_testbench.sv
module config_registers_testbench;

  // Testbench signals
  logic        clk;
  logic        rst_n;
  logic        write_enable;
  logic [31:0] config_data_in;
  logic [31:0] config_data_out;
  logic        interrupt_enable;
  logic [2:0]  priority_level;
  logic [7:0]  device_id;

  // Instantiate design under test
  hardware_config_registers CONFIG_DUT (
    .clk(clk),
    .rst_n(rst_n),
    .write_enable(write_enable),
    .config_data_in(config_data_in),
    .config_data_out(config_data_out),
    .interrupt_enable(interrupt_enable),
    .priority_level(priority_level),
    .device_id(device_id)
  );

  // Clock generation
  initial begin
    clk = 0;
    forever #5 clk = ~clk;  // 10ns period clock
  end

  // Test sequence
  initial begin
    // Dump waves
    $dumpfile("config_registers_testbench.vcd");
    $dumpvars(0, config_registers_testbench);

    // Initialize signals
    rst_n = 0;
    write_enable = 0;
    config_data_in = 32'h0;

    $display("=== Configuration Registers Test ===");
    $display();

    // Reset sequence
    #15;
    rst_n = 1;
    #10;

    $display("After Reset:");
    $display("  Device ID: 0x%02X", device_id);
    $display("  Priority:  %0d", priority_level);
    $display("  Int Enable: %b", interrupt_enable);
    $display("  Full Config: 0x%08X", config_data_out);
    $display();

    // Test configuration write 1: Enable interrupts with high priority
    #10;
    config_data_in = 32'h00FF_0701;  // DevID=0xFF, Priority=7, IntEn=1
    write_enable = 1;
    #10;
    write_enable = 0;
    #10;

    $display("Config Write 1 (High Priority + Interrupt Enable):");
    $display("  Device ID: 0x%02X", device_id);
    $display("  Priority:  %0d", priority_level);
    $display("  Int Enable: %b", interrupt_enable);
    $display("  Full Config: 0x%08X", config_data_out);
    $display();

    // Test configuration write 2: Different device ID, medium priority
    #10;
    config_data_in = 32'h0042_0420;  // DevID=0x42, Priority=4, IntEn=0
    write_enable = 1;
    #10;
    write_enable = 0;
    #10;

    $display("Config Write 2 (Medium Priority + Interrupt Disable):");
    $display("  Device ID: 0x%02X", device_id);
    $display("  Priority:  %0d", priority_level);
    $display("  Int Enable: %b", interrupt_enable);
    $display("  Full Config: 0x%08X", config_data_out);
    $display();

    // Test configuration write 3: Maximum values
    #10;
    config_data_in = 32'hABCD_EF01;  // Mixed values
    write_enable = 1;
    #10;
    write_enable = 0;
    #10;

    $display("Config Write 3 (Mixed Values):");
    $display("  Device ID: 0x%02X", device_id);
    $display("  Priority:  %0d", priority_level);
    $display("  Int Enable: %b", interrupt_enable);
    $display("  Full Config: 0x%08X", config_data_out);
    $display();

    // Test reset again
    #10;
    rst_n = 0;
    #10;
    rst_n = 1;
    #10;

    $display("After Second Reset:");
    $display("  Device ID: 0x%02X", device_id);
    $display("  Priority:  %0d", priority_level);
    $display("  Int Enable: %b", interrupt_enable);
    $display("  Full Config: 0x%08X", config_data_out);
    $display();

    $display("=== Configuration Registers Test Complete ===");
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Configuration Registers Test ===

After Reset:
  Device ID: 0xa5
  Priority:  2
  Int Enable: 0
  Full Config: 0x00a50202

Config Update: Device=0xa5, Priority=2, IntEn=0
Config Write 1 (High Priority + Interrupt Enable):
  Device ID: 0xff
  Priority:  7
  Int Enable: 1
  Full Config: 0x00ff0701

Config Update: Device=0xff, Priority=7, IntEn=1
Config Write 2 (Medium Priority + Interrupt Disable):
  Device ID: 0x42
  Priority:  4
  Int Enable: 0
  Full Config: 0x00420420

Config Update: Device=0x42, Priority=4, IntEn=0
Config Write 3 (Mixed Values):
  Device ID: 0xcd
  Priority:  7
  Int Enable: 1
  Full Config: 0xabcdef01

After Second Reset:
  Device ID: 0xa5
  Priority:  2
  Int Enable: 0
  Full Config: 0x00a50202

=== Configuration Registers Test Complete ===
Process finished with return code: 0
Removing Chapter_8_examples/example_19__config_registers/obj_dir directory...
Chapter_8_examples/example_19__config_registers/obj_dir removed successfully.


0

### **Example 20: Network Message**
Structure with fixed header fields and variable-length payload using queues.

In [4]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_20__network_message/"
read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// network_message_processor.sv
module network_message_processor;

  // Fixed header structure
  typedef struct {
    bit [7:0]  version;
    bit [7:0]  msg_type;
    bit [15:0] payload_length;
    bit [31:0] sequence_id;
  } message_header_t;

  // Complete message structure with header and payload queue
  typedef struct {
    message_header_t header;
    bit [7:0] payload_queue[$];
  } network_message_t;

  // Message processing queues
  network_message_t incoming_messages[$];
  network_message_t processed_messages[$];

  // Task to create a network message
  task create_message(input bit [7:0]  version,
                     input bit [7:0]  msg_type,
                     input bit [31:0] seq_id,
                     input bit [7:0]  payload_data[]);
    network_message_t new_msg;
    
    // Setup fixed header fields
    new_msg.header.version = version;
    new_msg.header.msg_type = msg_type;
    new_msg.header.sequence_id = seq_id;
    
    // Add variable-length payload to queue
    new_msg.payload_queue = {};
    foreach (payload_data[i]) begin
      new_msg.payload_queue.push_back(payload_data[i]);
    end
    new_msg.header.payload_length = new_msg.payload_queue.size()[15:0];
    
    // Add to incoming message queue
    incoming_messages.push_back(new_msg);
    
    $display("Created message: Type=%0d, Seq=%0d, PayloadLen=%0d",
             new_msg.header.msg_type,
             new_msg.header.sequence_id,
             new_msg.header.payload_length);
  endtask

  // Task to process incoming messages
  task process_messages();
    network_message_t current_msg;
    
    while (incoming_messages.size() > 0) begin
      current_msg = incoming_messages.pop_front();
      
      $display("Processing message:");
      $display("  Header - Ver:%0d Type:%0d Seq:%0d PayloadLen:%0d",
               current_msg.header.version,
               current_msg.header.msg_type,
               current_msg.header.sequence_id,
               current_msg.header.payload_length);
      
      $write("  Payload: ");
      foreach (current_msg.payload_queue[i]) begin
        $write("%02h ", current_msg.payload_queue[i]);
      end
      $display();
      
      // Move to processed queue
      processed_messages.push_back(current_msg);
    end
  endtask

  initial begin
    bit [7:0] test_payload1[] = '{8'hAA, 8'hBB, 8'hCC};
    bit [7:0] test_payload2[] = '{8'h11, 8'h22, 8'h33, 8'h44, 8'h55};
    bit [7:0] test_payload3[] = '{8'hFF};
    
    $display("=== Network Message Processor Demo ===");
    $display();
    
    // Create various messages with different payload lengths
    create_message(8'h01, 8'h10, 32'h1001, test_payload1);
    create_message(8'h01, 8'h20, 32'h1002, test_payload2);
    create_message(8'h01, 8'h30, 32'h1003, test_payload3);
    
    $display();
    $display("Messages in queue: %0d", incoming_messages.size());
    $display();
    
    // Process all messages
    process_messages();
    
    $display();
    $display("Processed messages: %0d", processed_messages.size());
    $display("Remaining incoming: %0d", incoming_messages.size());
  end

endmodule
```

```systemverilog
// network_message_processor_testbench.sv
module network_message_testbench;

  // Instantiate the network message processor
  network_message_processor MESSAGE_PROCESSOR_INSTANCE();

  initial begin
    // Dump waves for debugging
    $dumpfile("network_message_testbench.vcd");
    $dumpvars(0, network_message_testbench);
    
    $display("=== Network Message Testbench Started ===");
    $display();
    
    // Wait for the processor to complete
    #10;
    
    $display();
    $display("=== Network Message Testbench Completed ===");
    
    // Finish simulation
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Network Message Processor Demo ===

Created message: Type=16, Seq=4097, PayloadLen=3
Created message: Type=32, Seq=4098, PayloadLen=5
Created message: Type=48, Seq=4099, PayloadLen=1

Messages in queue: 3

Processing message:
  Header - Ver:1 Type:16 Seq:4097 PayloadLen:3
  Payload: aa bb cc
Processing message:
  Header - Ver:1 Type:32 Seq:4098 PayloadLen:5
  Payload: 11 22 33 44 55
Processing message:
  Header - Ver:1 Type:48 Seq:4099 PayloadLen:1
  Payload: ff

Processed messages: 3
Remaining incoming: 0
=== Network Message Testbench Started ===


=== Network Message Testbench Completed ===
Process finished with return code: 0
Removing Chapter_8_examples/example_20__network_message/obj_dir directory...
Chapter_8_examples/example_20__network_message/obj_dir removed successfully.


0

## Tagged Unions and Advanced Types

```systemverilog
// Tagged union for different data types
typedef union tagged {
    void        Invalid;
    bit [7:0]   Byte;
    bit [15:0]  Word;
    bit [31:0]  DWord;
    real        Float;
    string      Text;
} variant_data_t;

// Generic container structure
typedef struct {
    string name;
    variant_data_t data;
    int timestamp;
} data_container_t;

module tagged_unions;
    variant_data_t var_data;
    data_container_t containers[$];
    
    initial begin
        // Create different data containers
        data_container_t container;
        
        // Byte data
        container.name = "Status";
        container.data = tagged Byte (8'hAA);
        container.timestamp = $time;
        containers.push_back(container);
        
        // Word data
        container.name = "Port";
        container.data = tagged Word (16'h1234);
        container.timestamp = $time + 10;
        containers.push_back(container);
        
        // DWord data
        container.name = "Address";
        container.data = tagged DWord (32'hDEADBEEF);
        container.timestamp = $time + 20;
        containers.push_back(container);
        
        // Float data
        container.name = "Temperature";
        container.data = tagged Float (23.5);
        container.timestamp = $time + 30;
        containers.push_back(container);
        
        // String data
        container.name = "Message";
        container.data = tagged Text ("Hello World");
        container.timestamp = $time + 40;
        containers.push_back(container);
        
        // Process all containers
        foreach(containers[i]) begin
            $display("\nContainer %0d: %s", i, containers[i].name);
            $display("  Timestamp: %0t", containers[i].timestamp);
            
            case (containers[i].data) matches
                tagged Invalid:           $display("  Data: Invalid");
                tagged Byte .b:          $display("  Data: Byte = 0x%02h", b);
                tagged Word .w:          $display("  Data: Word = 0x%04h", w);
                tagged DWord .dw:        $display("  Data: DWord = 0x%08h", dw);
                tagged Float .f:         $display("  Data: Float = %f", f);
                tagged Text .s:          $display("  Data: Text = '%s'", s);
            endcase
        end
    end
endmodule
```

### **Example 21: Multi-Type Container**
Tagged union that can hold different data types (integer, string, float) safely.

In [11]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_21__multi_type_container/"
read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// multi_type_container.sv
module multi_type_container;

  // Enumeration for data types
  typedef enum logic [1:0] {
    TYPE_INTEGER = 2'b00,
    TYPE_STRING  = 2'b01,
    TYPE_FLOAT   = 2'b10,
    TYPE_EMPTY   = 2'b11
  } data_type_e;

  // Union for different data types
  typedef union packed {
    logic [63:0] integer_value;  // 64-bit to match union size
    logic [63:0] string_hash;    // 64-bit to match union size
    logic [63:0] float_bits;     // Store float as 64-bit pattern to match $realtobits
  } data_value_u;

  // Container structure
  typedef struct {
    data_type_e  data_type;
    data_value_u data_value;
    string       string_data; // Keep actual string separately
    real         float_data;  // Keep actual float separately
  } data_container_t;

  // Container instance
  data_container_t container;

  // Initialize container
  initial begin
    container.data_type = TYPE_EMPTY;
    container.string_data = "";
    container.float_data = 0.0;
  end

  // Function to store integer value
  function void store_integer(int value);
    container.data_type = TYPE_INTEGER;
    container.data_value.integer_value = 64'(value); // Explicit cast to 64-bit
    $display("Stored integer: %0d", value);
  endfunction

  // Function to store string value
  function void store_string(string value);
    container.data_type = TYPE_STRING;
    container.string_data = value;
    container.data_value.string_hash = 64'(value.len()); // Explicit cast to 64-bit
    $display("Stored string: %s", value);
  endfunction

  // Function to store float value
  function void store_float(real value);
    container.data_type = TYPE_FLOAT;
    container.float_data = value;
    container.data_value.float_bits = $realtobits(value);
    $display("Stored float: %0.2f", value);
  endfunction

  // Function to safely retrieve and display current value
  function void display_container();
    case (container.data_type)
      TYPE_INTEGER: begin
        $display("Container holds integer: %0d", 
                 int'(container.data_value.integer_value)); // Cast back to int for display
      end
      TYPE_STRING: begin
        $display("Container holds string: %s", container.string_data);
      end
      TYPE_FLOAT: begin
        $display("Container holds float: %0.2f", container.float_data);
      end
      TYPE_EMPTY: begin
        $display("Container is empty");
      end
      default: begin
        $display("Container holds unknown type");
      end
    endcase
  endfunction

  // Function to get container type as string
  function string get_container_type();
    case (container.data_type)
      TYPE_INTEGER: return "integer";
      TYPE_STRING:  return "string";
      TYPE_FLOAT:   return "float";
      TYPE_EMPTY:   return "empty";
      default:      return "unknown";
    endcase
  endfunction

  // Function to check if container is empty
  function bit is_empty();
    return (container.data_type == TYPE_EMPTY);
  endfunction

  // Function to clear container
  function void clear_container();
    container.data_type = TYPE_EMPTY;
    container.string_data = "";
    container.float_data = 0.0;
    $display("Container cleared");
  endfunction

endmodule
```

```systemverilog
// multi_type_container_testbench.sv
module multi_type_container_testbench;

  // Instantiate the multi-type container
  multi_type_container CONTAINER_INSTANCE();

  initial begin
    // Dump waves
    $dumpfile("multi_type_container_testbench.vcd");
    $dumpvars(0, multi_type_container_testbench);
    
    $display("=== Multi-Type Container Example ===");
    $display();

    // Test storing different types of data
    $display("--- Testing Integer Storage ---");
    CONTAINER_INSTANCE.store_integer(42);
    CONTAINER_INSTANCE.display_container();
    $display("Type: %s", CONTAINER_INSTANCE.get_container_type());
    $display();

    $display("--- Testing String Storage ---");
    CONTAINER_INSTANCE.store_string("Hello SystemVerilog!");
    CONTAINER_INSTANCE.display_container();
    $display("Type: %s", CONTAINER_INSTANCE.get_container_type());
    $display();

    $display("--- Testing Float Storage ---");
    CONTAINER_INSTANCE.store_float(3.14159);
    CONTAINER_INSTANCE.display_container();
    $display("Type: %s", CONTAINER_INSTANCE.get_container_type());
    $display();

    $display("--- Testing Type Safety ---");
    CONTAINER_INSTANCE.store_integer(-100);
    $display("After storing new integer:");
    CONTAINER_INSTANCE.display_container();
    $display("Type: %s", CONTAINER_INSTANCE.get_container_type());
    $display();

    $display("--- Testing Container State Management ---");
    $display("Is container empty? %s", 
             CONTAINER_INSTANCE.is_empty() ? "Yes" : "No");
    
    CONTAINER_INSTANCE.clear_container();
    CONTAINER_INSTANCE.display_container();
    $display("Is container empty? %s", 
             CONTAINER_INSTANCE.is_empty() ? "Yes" : "No");
    $display();

    $display("--- Final Test ---");
    CONTAINER_INSTANCE.store_string("Final test complete!");
    CONTAINER_INSTANCE.display_container();
    $display("Type: %s", CONTAINER_INSTANCE.get_container_type());
    $display();

    $display("=== Test Complete ===");
    
    #1; // Small delay
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Multi-Type Container Example ===

--- Testing Integer Storage ---
Stored integer: 42
Container holds integer: 42
Type: integer

--- Testing String Storage ---
Stored string: Hello SystemVerilog!
Container holds string: Hello SystemVerilog!
Type: string

--- Testing Float Storage ---
Stored float: 3.14
Container holds float: 3.14
Type: float

--- Testing Type Safety ---
Stored integer: -100
After storing new integer:
Container holds integer: -100
Type: integer

--- Testing Container State Management ---
Is container empty?  No
Container cleared
Container is empty
Is container empty? Yes

--- Final Test ---
Stored string: Final test complete!
Container holds string: Final test complete!
Type: string

=== Test Complete ===
Process finished with return code: 0
Removing Chapter_8_examples/example_21__multi_type_container/obj_dir directory...
Chapter_8_examples/example_21__multi_type_container/obj_dir removed successfully.


0

### **Example 22: Protocol Packet Parser**
Using tagged unions to handle different packet types in a communication protocol.

In [16]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_22__protocol_packet_parser/"
read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// protocol_packet_parser.sv
module protocol_packet_parser;

  // Define packet types using enum
  typedef enum logic [1:0] {
    DATA_PACKET  = 2'b00,
    ACK_PACKET   = 2'b01,
    ERROR_PACKET = 2'b10,
    RESET_PACKET = 2'b11
  } packet_type_t;

  // Define structures for different packet data (all 24 bits)
  typedef struct packed {
    logic [7:0]  sequence_num;
    logic [15:0] data_payload;
  } data_packet_s;

  typedef struct packed {
    logic [7:0]  ack_num;
    logic        status;
    logic [14:0] reserved; // Padding to make 24 bits total
  } ack_packet_s;

  typedef struct packed {
    logic [7:0]  error_code;
    logic [15:0] reserved;   // Padding to make 24 bits total
  } error_packet_s;

  // Main packet structure
  typedef struct packed {
    packet_type_t packet_type;
    logic [23:0]  packet_data; // Union-like data field
  } protocol_packet_t;

  // Function to create a data packet
  function protocol_packet_t create_data_packet(logic [7:0] seq_num, 
                                                logic [15:0] data);
    protocol_packet_t packet;
    data_packet_s data_struct;
    
    data_struct.sequence_num = seq_num;
    data_struct.data_payload = data;
    packet.packet_type = DATA_PACKET;
    packet.packet_data = data_struct;
    return packet;
  endfunction

  // Function to create an ACK packet
  function protocol_packet_t create_ack_packet(logic [7:0] ack_num, 
                                               logic status);
    protocol_packet_t packet;
    ack_packet_s ack_struct;
    
    ack_struct.ack_num = ack_num;
    ack_struct.status = status;
    ack_struct.reserved = 15'b0;
    packet.packet_type = ACK_PACKET;
    packet.packet_data = ack_struct;
    return packet;
  endfunction

  // Function to create an error packet
  function protocol_packet_t create_error_packet(logic [7:0] error_code);
    protocol_packet_t packet;
    error_packet_s error_struct;
    
    error_struct.error_code = error_code;
    error_struct.reserved = 16'b0;
    packet.packet_type = ERROR_PACKET;
    packet.packet_data = error_struct;
    return packet;
  endfunction

  // Function to create a reset packet
  function protocol_packet_t create_reset_packet();
    protocol_packet_t packet;
    packet.packet_type = RESET_PACKET;
    packet.packet_data = 24'b0;
    return packet;
  endfunction

  // Function to parse and display packet information
  function void parse_packet(protocol_packet_t packet);
    case (packet.packet_type)
      DATA_PACKET: begin
        data_packet_s data_info;
        data_info = data_packet_s'(packet.packet_data[23:0]);
        $display("DATA PACKET:");
        $display("  Sequence Number: %0d", data_info.sequence_num);
        $display("  Data Payload: 0x%04h", data_info.data_payload);
      end
      
      ACK_PACKET: begin
        ack_packet_s ack_info;
        ack_info = ack_packet_s'(packet.packet_data[23:0]);
        $display("ACK PACKET:");
        $display("  ACK Number: %0d", ack_info.ack_num);
        $display("  Status: %s", ack_info.status ? "SUCCESS" : "FAIL");
      end
      
      ERROR_PACKET: begin
        logic [7:0] error_code;
        error_code = packet.packet_data[23:16];
        $display("ERROR PACKET:");
        $display("  Error Code: %0d", error_code);
      end
      
      RESET_PACKET: begin
        $display("RESET PACKET: System reset requested");
      end
      
      default: begin
        $display("UNKNOWN PACKET TYPE");
      end
    endcase
    $display("----------------------------------------");
  endfunction

  // Function to get packet type as string
  function string get_packet_type_string(protocol_packet_t packet);
    case (packet.packet_type)
      DATA_PACKET:  return "DATA";
      ACK_PACKET:   return "ACK";
      ERROR_PACKET: return "ERROR";
      RESET_PACKET: return "RESET";
      default:      return "UNKNOWN";
    endcase
  endfunction

  // Function to validate packet integrity
  function bit is_valid_packet(protocol_packet_t packet);
    case (packet.packet_type)
      DATA_PACKET, ACK_PACKET, ERROR_PACKET, RESET_PACKET: return 1'b1;
      default: return 1'b0;
    endcase
  endfunction

endmodule
```

```systemverilog
// protocol_packet_parser_testbench.sv
module packet_parser_testbench;

  // Instantiate the design under test
  protocol_packet_parser PARSER_INSTANCE();

  // Import types from the design (Verilator-compatible approach)
  typedef enum logic [1:0] {
    DATA_PACKET  = 2'b00,
    ACK_PACKET   = 2'b01,
    ERROR_PACKET = 2'b10,
    RESET_PACKET = 2'b11
  } packet_type_t;

  typedef struct packed {
    packet_type_t packet_type;
    logic [23:0]  packet_data;
  } protocol_packet_t;

  // Test variables
  protocol_packet_t test_packet;

  initial begin
    // Dump waves
    $dumpfile("packet_parser_testbench.vcd");
    $dumpvars(0, packet_parser_testbench);

    $display("=== Protocol Packet Parser Test ===");
    $display();

    // Test 1: Data packet
    $display("Test 1: Creating and parsing DATA packet");
    test_packet = PARSER_INSTANCE.create_data_packet(8'd42, 16'hABCD);
    PARSER_INSTANCE.parse_packet(test_packet);
    $display("Packet type: %s", 
             PARSER_INSTANCE.get_packet_type_string(test_packet));
    $display("Valid packet: %s", 
             PARSER_INSTANCE.is_valid_packet(test_packet) ? "YES" : "NO");
    $display();

    // Test 2: ACK packet with success status
    $display("Test 2: Creating and parsing ACK packet (SUCCESS)");
    test_packet = PARSER_INSTANCE.create_ack_packet(8'd42, 1'b1);
    PARSER_INSTANCE.parse_packet(test_packet);
    $display("Packet type: %s", 
             PARSER_INSTANCE.get_packet_type_string(test_packet));
    $display("Valid packet: %s", 
             PARSER_INSTANCE.is_valid_packet(test_packet) ? "YES" : "NO");
    $display();

    // Test 3: ACK packet with fail status
    $display("Test 3: Creating and parsing ACK packet (FAIL)");
    test_packet = PARSER_INSTANCE.create_ack_packet(8'd15, 1'b0);
    PARSER_INSTANCE.parse_packet(test_packet);
    $display("Packet type: %s", 
             PARSER_INSTANCE.get_packet_type_string(test_packet));
    $display("Valid packet: %s", 
             PARSER_INSTANCE.is_valid_packet(test_packet) ? "YES" : "NO");
    $display();

    // Test 4: Error packet
    $display("Test 4: Creating and parsing ERROR packet");
    test_packet = PARSER_INSTANCE.create_error_packet(8'd255);
    PARSER_INSTANCE.parse_packet(test_packet);
    $display("Packet type: %s", 
             PARSER_INSTANCE.get_packet_type_string(test_packet));
    $display("Valid packet: %s", 
             PARSER_INSTANCE.is_valid_packet(test_packet) ? "YES" : "NO");
    $display();

    // Test 5: Reset packet
    $display("Test 5: Creating and parsing RESET packet");
    test_packet = PARSER_INSTANCE.create_reset_packet();
    PARSER_INSTANCE.parse_packet(test_packet);
    $display("Packet type: %s", 
             PARSER_INSTANCE.get_packet_type_string(test_packet));
    $display("Valid packet: %s", 
             PARSER_INSTANCE.is_valid_packet(test_packet) ? "YES" : "NO");
    $display();

    // Test 6: Demonstrate packet type checking
    $display("Test 6: Packet type verification");
    test_packet = PARSER_INSTANCE.create_data_packet(8'd10, 16'h1234);
    
    if (test_packet.packet_type == DATA_PACKET)
      $display("Correctly identified as DATA packet");
    else
      $display("Failed to identify DATA packet");

    if (test_packet.packet_type == ACK_PACKET)
      $display("Incorrectly identified as ACK packet");
    else
      $display("Correctly rejected as ACK packet");

    // Test 7: Manual packet creation (testing direct assignment)
    $display();
    $display("Test 7: Testing manual packet creation");
    test_packet.packet_type = RESET_PACKET; // Use enum value
    test_packet.packet_data = 24'hDEADBE;
    $display("Manual packet type: %s", 
             PARSER_INSTANCE.get_packet_type_string(test_packet));
    $display("Valid packet: %s", 
             PARSER_INSTANCE.is_valid_packet(test_packet) ? "YES" : "NO");
    PARSER_INSTANCE.parse_packet(test_packet);

    $display();
    $display("=== All tests completed ===");
    
    #10 $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Protocol Packet Parser Test ===

Test 1: Creating and parsing DATA packet
DATA PACKET:
  Sequence Number: 42
  Data Payload: 0xabcd
----------------------------------------
Packet type: DATA
Valid packet: YES

Test 2: Creating and parsing ACK packet (SUCCESS)
ACK PACKET:
  ACK Number: 42
  Status: SUCCESS
----------------------------------------
Packet type: ACK
Valid packet: YES

Test 3: Creating and parsing ACK packet (FAIL)
ACK PACKET:
  ACK Number: 15
  Status:    FAIL
----------------------------------------
Packet type: ACK
Valid packet: YES

Test 4: Creating and parsing ERROR packet
ERROR PACKET:
  Error Code: 255
----------------------------------------
Packet type: ERROR
Valid packet: YES

Test 5: Creating and parsing RESET packet
RESET PACKET: System reset requested
----------------------------------------
Packet type: RESET
Valid packet: YES

Test 6: Packet type verification
Correctly identified as DATA packet
Correctly rejected as ACK packet

0

### **Example 23: Generic Data Storage**
Container structure using tagged unions for storing various data formats with metadata.

In [19]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_8_examples/example_23__generic_data_storage/"
read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// generic_data_container.sv
// Generic data storage using tagged unions for various data formats

module data_container_module ();

  // Define data types that can be stored
  typedef enum {
    DATA_INTEGER,
    DATA_REAL,
    DATA_STRING,
    DATA_BITS
  } data_type_t;

  // Union for storing different data formats
  typedef union packed {
    int       integer_data;       // Integer storage
    bit [31:0] bits_data;        // Bit vector storage
  } storage_data_t;
  
  // Separate storage for non-packed types
  typedef struct {
    real      real_value;         // Real number storage
    string    string_value;       // String storage
  } extended_data_t;

  // Container structure with metadata
  typedef struct {
    data_type_t     data_type;      // Type identifier
    string          label;          // Data label/name
    time            timestamp;      // Creation timestamp
    storage_data_t  data;          // Packed data (int/bits)
    extended_data_t ext_data;      // Extended data (real/string)
  } data_container_t;

  // Storage array for multiple containers
  data_container_t storage_array[10];
  int storage_count = 0;

  // Function to store integer data
  function void store_integer(string label_name, int value);
    if (storage_count < 10) begin
      storage_array[storage_count].data_type = DATA_INTEGER;
      storage_array[storage_count].label = label_name;
      storage_array[storage_count].timestamp = $time;
      storage_array[storage_count].data.integer_data = value;
      storage_count++;
      $display("Stored integer: %s = %0d at time %0t", 
               label_name, value, $time);
    end
  endfunction

  // Function to store real data
  function void store_real(string label_name, real value);
    if (storage_count < 10) begin
      storage_array[storage_count].data_type = DATA_REAL;
      storage_array[storage_count].label = label_name;
      storage_array[storage_count].timestamp = $time;
      storage_array[storage_count].ext_data.real_value = value;
      storage_count++;
      $display("Stored real: %s = %0.2f at time %0t", 
               label_name, value, $time);
    end
  endfunction

  // Function to store string data
  function void store_string(string label_name, string value);
    if (storage_count < 10) begin
      storage_array[storage_count].data_type = DATA_STRING;
      storage_array[storage_count].label = label_name;
      storage_array[storage_count].timestamp = $time;
      storage_array[storage_count].ext_data.string_value = value;
      storage_count++;
      $display("Stored string: %s = '%s' at time %0t", 
               label_name, value, $time);
    end
  endfunction

  // Function to store bit vector data
  function void store_bits(string label_name, bit [31:0] value);
    if (storage_count < 10) begin
      storage_array[storage_count].data_type = DATA_BITS;
      storage_array[storage_count].label = label_name;
      storage_array[storage_count].timestamp = $time;
      storage_array[storage_count].data.bits_data = value;
      storage_count++;
      $display("Stored bits: %s = 0x%08h at time %0t", 
               label_name, value, $time);
    end
  endfunction

  // Function to retrieve and display stored data
  function void display_storage();
    $display("\n=== Data Container Storage Report ===");
    for (int i = 0; i < storage_count; i++) begin
      $display("Index %0d: Label='%s', Time=%0t", 
               i, storage_array[i].label, storage_array[i].timestamp);
      
      case (storage_array[i].data_type)
        DATA_INTEGER: begin
          $display("  Type: INTEGER, Value: %0d", 
                   storage_array[i].data.integer_data);
        end
        DATA_REAL: begin
          $display("  Type: REAL, Value: %0.2f", 
                   storage_array[i].ext_data.real_value);
        end
        DATA_STRING: begin
          $display("  Type: STRING, Value: '%s'", 
                   storage_array[i].ext_data.string_value);
        end
        DATA_BITS: begin
          $display("  Type: BITS, Value: 0x%08h", 
                   storage_array[i].data.bits_data);
        end
      endcase
      $display();
    end
  endfunction

  initial begin
    $display("Generic Data Storage Container Example");
    $display("=====================================");
    
    // Store different types of data
    #10 store_integer("counter_value", 42);
    #10 store_real("voltage_level", 3.14);
    #10 store_string("device_name", "FPGA_Board");
    #10 store_bits("control_reg", 32'hDEADBEEF);
    #10 store_integer("temperature", -25);
    
    // Display all stored data
    #10 display_storage();
  end

endmodule
```

```systemverilog
// generic_data_container_testbench.sv
// Testbench for generic data storage container

module data_container_testbench;

  // Instantiate the design under test
  data_container_module CONTAINER_INSTANCE();

  // Additional test scenarios
  initial begin
    // Dump waves for debugging
    $dumpfile("data_container_testbench.vcd");
    $dumpvars(0, data_container_testbench);
    
    $display("\n=== Testbench: Additional Storage Operations ===");
    
    // Wait for initial operations to complete
    #100;
    
    // Test storage of additional data types
    CONTAINER_INSTANCE.store_string("project_id", "SV_Example_23");
    #5 CONTAINER_INSTANCE.store_bits("status_flags", 32'b10101010);
    #5 CONTAINER_INSTANCE.store_real("frequency", 100.5);
    
    // Display updated storage
    #10 CONTAINER_INSTANCE.display_storage();
    
    // Test storage limit (array is size 10)
    $display("\n=== Testing Storage Limit ===");
    CONTAINER_INSTANCE.store_integer("overflow_test", 999);
    CONTAINER_INSTANCE.store_integer("overflow_test2", 1000);
    
    #10 $display("\nTotal items stored: %0d", 
                 CONTAINER_INSTANCE.storage_count);
    
    #10 $finish;
  end

  // Monitor storage operations
  initial begin
    $monitor("Time %0t: Storage count = %0d", 
             $time, CONTAINER_INSTANCE.storage_count);
  end

endmodule
```

Verilator Simulation Output:

=== Testbench: Additional Storage Operations ===
Generic Data Storage Container Example
Time 0: Storage count = 0
Stored integer: counter_value = 42 at time 10
Time 10: Storage count = 1
Stored real: voltage_level = 3.14 at time 20
Time 20: Storage count = 2
Stored string: device_name = 'FPGA_Board' at time 30
Time 30: Storage count = 3
Stored bits: control_reg = 0xdeadbeef at time 40
Time 40: Storage count = 4
Stored integer: temperature = -25 at time 50
Time 50: Storage count = 5

=== Data Container Storage Report ===
Index 0: Label='counter_value', Time=10
  Type: INTEGER, Value: 42

Index 1: Label='voltage_level', Time=20
  Type: REAL, Value: 3.14

Index 2: Label='device_name', Time=30
  Type: STRING, Value: 'FPGA_Board'

Index 3: Label='control_reg', Time=40
  Type: BITS, Value: 0xdeadbeef

Index 4: Label='temperature', Time=50
  Type: INTEGER, Value: -25

Stored string: project_id = 'SV_Example_23' at time 100
Time 100: Storage count = 6
Stored bits:

0

## Best Practices

When working with advanced data types in SystemVerilog:
1. **Choose the Right Type**: Use dynamic arrays for variable-size data, associative arrays for sparse data, and queues for ordered collections.
2. **Memory Management**: Always consider memory allocation and deallocation, especially with dynamic arrays.
3. **Type Safety**: Use typedef and enumerations to create self-documenting, type-safe code.
4. **Packed vs Unpacked**: Use packed structures/arrays when you need bit-level access or specific memory layout.
5. **Performance Considerations**: Associative arrays have lookup overhead; use regular arrays when performance is critical and size is known.
6. **Code Organization**: Group related data using structures and use unions when you need different interpretations of the same data.